In [2]:
import pandas as pd
import os
import seaborn as sns
from datetime import datetime
from fpdf import FPDF
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

# === CONFIGURATION ===
folder_path = '/Users/ajfoeckler/Downloads/chicago_games'
save_folder_path = '/Users/ajfoeckler/Downloads/chicago_pitching'
selected_team = 'CHI_DOG'


os.makedirs(save_folder_path, exist_ok=True)
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

data_frames = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    data_frames.append(df)

original_data = pd.concat(data_frames, ignore_index=True)
original_data['Date'] = pd.to_datetime(original_data['Date'], errors='coerce')
original_data = original_data.dropna(subset=['Date'])
original_data = original_data[original_data['Date'].dt.year == 2025]

team_pitchers = original_data[original_data['PitcherTeam'] == selected_team].copy()

pitch_type_map = {
    '4-Seam': 'Four-Seam',
    '4-Seam Fastball': 'Four-Seam',
    'Fastball': 'Four-Seam',
    'Four-seam': 'Four-Seam',
    'Four seam': 'Four-Seam',
    '2-Seam Fastball': 'Sinker',
    'Sinker': 'Sinker',
    'Slider': 'Slider',
    'Curveball': 'Curveball',
    'Changeup': 'Changeup',
    'Cutter': 'Cutter',
    'Splitter': 'Splitter',
    'Knuckleball': 'Knuckleball'
}

team_pitchers['TaggedPitchType_clean'] = team_pitchers['AutoPitchType'].astype(str).str.strip().str.lower()
pitch_type_map_cleaned = {k.lower(): v for k, v in pitch_type_map.items()}
team_pitchers['TaggedPitchType'] = team_pitchers['TaggedPitchType_clean'].map(pitch_type_map_cleaned)
team_pitchers.loc[
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['HorzBreak'] > 10), 'TaggedPitchType'] = 'Sweeper'
threshold = -7.5
mask_bourassa = (
    (team_pitchers['Pitcher'] == 'Bourassa, Landen') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > threshold)
)
print(f"Auto-retagging {mask_bourassa.sum()} Bourassa curveballs → sliders (IVB > {threshold})")
team_pitchers.loc[mask_bourassa, 'TaggedPitchType'] = 'Slider'
     # —— Strobel Tasker: auto-retag offspeed for his profile ——
     # 1) sliders with extra horizontal break become sweepers
mask_sweeper = (      
         (team_pitchers['Pitcher'] == 'Strobel, Tasker') &
         (team_pitchers['TaggedPitchType'] == 'Slider') &
         (team_pitchers['HorzBreak'] > 10)
     )
print(f"Strobel: retagging {mask_sweeper.sum()} sliders → sweepers")
team_pitchers.loc[mask_sweeper, 'TaggedPitchType'] = 'Sweeper'
    # 2) shallower curveballs become sliders
mask_curve_to_slider = (
         (team_pitchers['Pitcher'] == 'Strobel, Tasker') &
         (team_pitchers['TaggedPitchType'] == 'Curveball') &
         (team_pitchers['InducedVertBreak'] > -3)
     )
print(f"Strobel: retagging {mask_curve_to_slider.sum()} curveballs → sliders")
team_pitchers.loc[mask_curve_to_slider, 'TaggedPitchType'] = 'Slider'
mask_galindo = (
    (team_pitchers['Pitcher'] == 'Galindo, Jesse') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] < 11)
)
print(f"Auto-retagging {mask_galindo.sum()} Galindo four-seams → sinkers (IVB < 11)")
team_pitchers.loc[mask_galindo, 'TaggedPitchType'] = 'Sinker'
mask_bourassa_change = (
    (team_pitchers['Pitcher'] == 'Bourassa, Landen') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto-retagging {mask_bourassa_change.sum()} Bourassa changeups → sinkers (RelSpeed > 85)")
team_pitchers.loc[mask_bourassa_change, 'TaggedPitchType'] = 'Sinker'
mask_cherry_cutter = (
    (team_pitchers['Pitcher'] == 'Cherry, Derrick') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] < 10)
)
print(f"Auto-retagging {mask_cherry_cutter.sum()} Cherry sinkers → cutters (HorzBreak < 10)")
team_pitchers.loc[mask_cherry_cutter, 'TaggedPitchType'] = 'Cutter'
mask_echev = (
    (team_pitchers['Pitcher'] == 'Echevarria, Juan') &
    (team_pitchers['TaggedPitchType'] == 'Curveball')
)
print(f"Auto-retagging {mask_echev.sum()} Echevarria curveballs → sweepers")
team_pitchers.loc[mask_echev, 'TaggedPitchType'] = 'Sweeper'
mask_lambson_slider = (
    (team_pitchers['Pitcher'] == 'Lambson, Mitchell') &
    (team_pitchers['TaggedPitchType'] == 'Slider')
)
team_pitchers.loc[
    mask_lambson_slider & (team_pitchers['RelSpeed'] > 75),
    'TaggedPitchType'
] = 'Four-Seam'
team_pitchers.loc[
    mask_lambson_slider & (team_pitchers['RelSpeed'] <= 75),
    'TaggedPitchType'
] = 'Changeup'
team_pitchers = team_pitchers[team_pitchers['TaggedPitchType'].notna()]
# ——— manual override mis-tags for specific pitchers ———
manual_overrides = {
    ('Baker, John', 'slider'): 'Cutter',
    ('Davidson, Zach', 'curveball'): 'Slider',
    ('Keys, JC', 'cutter'): "Four-Seam",
    ('Curlis, Connor','sinker'): "Four-Seam",
    ('Puckett, Brady', 'slider'): "Cutter",
    ('Puckett, Brady', 'curveball'): "Slider",
    ('Bartow, Frankie', 'sinker'): "Four-Seam",
    ('Bentley, Denny', 'curveball'): "Curveball",
    ('Mendez, Isaac', 'sinker'): "Four-Seam",
    ('Davis, Colten', 'curveball'): "Slider",
    ('Dykhoff, Jake', 'sinker'): "Four-Seam",
    ('Harm, Parker', 'curveball'): "Sweeper",
    ('Rodriguez, Orlando', 'sinker'): "Four-Seam",
    ('Johnston, Kyle', 'curveball'): "Slider",
    ('Paulino, Naswell', 'cutter'): "Four-Seam",
    ('Quesada, Rogelio', 'curveball'): "Slider",
    ('Quesada, Rogelio', 'four-seam'): "Sinker",
    ('Minier, Greg', 'curveball'): "Curveball",
    ('Minier, Greg', 'sinker'): "Four-Seam",
    ('Brown, Tanner', 'curveball'): "Curveball",
    ('Cosby, Christian', 'changeup'): "Splitter",
    ('Dorminy, Thomas', 'curveball'): "Curveball",
    ('Dorminy, Thomas', 'slider'): "Cutter",
    ('Dorminy, Thomas', 'sinker'): "Four-Seam",
    ('Johnson, Christian', 'changeup'): "Splitter",
    ('Johnson, Christian', 'cutter'): "Four-Seam",
    ('Stover, Brady', 'cutter'): "Slider",
    ('Brentz, Jake', 'sinker'): "Four-Seam",
    ('Cerda, Junior', 'four-seam'): "Sinker",
    ('Cerda, Junior', 'curveball'): "Slider",
    ('Garcia, Julian', 'four-seam'): "Cutter",
    ('Garcia, Julian', 'curveball'): "Slider",
    ('Goddard, Jackson', 'curveball'): "Slider",
    ('Goldsberry, Blake', 'sinker'): "Four-Seam",
    ('Hakanson, Jeff', 'sinker'): "Four-Seam",
    ('Hakanson, Jeff', 'curveball'): "Sweeper",
    ('Hakanson, Jeff', 'slider'): "Sweeper",
    ('Hendrickson, Josh', 'sinker'): "Four-Seam",
    ('Hendrickson, Josh', 'slider'): "Cutter",
    ('Holden, Connor', 'curveball'): "Slider",
    ('Holden, Connor', 'sinker'): "Four-Seam",
    ('Mendez, Leam', 'sinker'): "Four-Seam",
    ('Mendez, Leam', 'curveball'): "Slider",
    ('Martinez, Daniel', 'slider'): "Cutter",
    ('McKay, Tyler', 'slider'): "Cutter",
    ('McKay, Tyler', 'curveball'): "Slider",
    ('Blain, Nate', 'curveball'): "Slider",
    ('Blain, Nate', 'sinker'): "Four-Seam",
    ('Blake, Johnny', 'four-seam'): "Sinker",
    ('Blake, Johnny', 'curveball'): "Slider",
    ('Blake, Johnny', 'changeup'): "Sinker",
    ('Cobos, Franny', 'sinker'): "Four-Seam",
    ('Loukinen, Greg', 'changeup'): "Sinker",
    ('Mullenbach, Matt', 'curveball'): "Slider",
    ('Loukinen, Greg', 'curveball'): "Slider",
    ('Loukinen, Greg', 'slider'): "Slider",
    ('Bourassa, Landen', 'four-seam'): "Sinker",
    ('Bourassa, Landen', 'slider'): "Cutter",
    ('Brigden, Trevor', 'slider'): "Cutter",
    ('Cherry, Derrick', 'slider'): "Cutter",
    ('Cherry, Derrick', 'sinker'): "Four-Seam",
    ('Echevarria, Juan', 'changeup'): "Sinker",
    ('Galindo, Jesse', 'changeup'): "Four-Seam",
    ('Lambson, Mitchell', 'curveball'): "Sweeper",
    ('Lambson, Mitchell', 'changeup'): "Four-Seam",
    ('Onyshko, Ben', 'curveball'): "Slider",
    ('Onyshko, Ben', 'changeup'): "Sinker",
    ('Sierra, Will', 'curveball'): "Slider",
    ('Yakel, Ryder', 'curveball'): "Slider",
    ('Strobel, Tasker', 'sinker'): "Four-Seam",
    ('Galino, Jesse', 'four-seam'): "Sinker",
    ('Bourassa, Landen', 'sinker'): "Changeup",
    ('Sierra, Will', 'sinker'): "Four-Seam",
    ('Madison, Ben', 'cutter'): "Four-Seam",
    ('Madison, Ben', 'curveball'): "Slider",
    ('Jessee, Chase', 'curveball'): "Curveball",
    ('Jessee, Chase', 'sinker'): "Four-Seam",
    ('Goins, Jeremy', 'sinker'): "Four-Seam",
    ('Goins, Jeremy', 'curveball'): "Slurve",
    ('Jackson, Jaren', 'slider'): "Sweeper",
    ('Gercken, Nate', 'curveball'): "Slider",
    ('Scholten, JD', 'changeup'): "Sinker",
    ('Scholten, JD', 'slider'): "Cutter",
    ('Scholten, JD', 'curveball'): "Slider",
    ('Willeman, Zach', 'sinker'): "Changeup",
    ('Willeman, Zach', 'cutter'): "Slider",
    ('Wetherbee, Jared', 'slider'): "Cutter",
    ('Wetherbee, Jared', 'curveball'): "Curveball",
    ('Crigger, Kyle', 'curveball'): "Slider",
    ('Harm, Parker', 'slider'): "Sweeper",
    ('Landis, Dutch', 'curveball'): "Slider",
    ('Langrell, Connor', 'cutter'): "Slider",
    ('Castaneda, Dylan', 'curveball'): "Slider",
    ('Bentley, Denny', 'sinker'): "Four-Seam",
    ('Hernandez, Nyan', 'sinker'): "Four-Seam",
    ('Mezquita, Jhordany', 'sinker'): "Four-Seam",
    ('Mezquita, Jhordany', 'curveball'): "Curveball",
    ('Mezquita, Jhordany', 'slider'): "Curveball",
    ('Snow, Logan', 'curveball'): "Slider",
    ('Thomas, Tahnaj', 'curveball'): "Slider",
    ('Thomas, Tahnaj', 'sinker'): "Four-Seam",
    ('Walker, Matt', 'sinker'): "Four-Seam",
    ('Welch, Davis', 'curveball'): "Slider",
    ('Craft, Derek', 'curveball'): "Slider",
    ('Faith, Austin', 'curveball'): "Slider",
    ('Faith, Austin', 'sinker'): "Four-Seam",
    ('McDowell, Theo', 'sinker'): "Four-Seam",
    ('McDowell, Theo', 'changeup'): "Four-Seam",
    ('Rivera, Jerryell', 'cutter'): "Four-Seam",
    ('Reininger, Zac', 'four-seam'): "Sinker",
    ('Reininger, Zac', 'slider'): "Cutter",
    ('Manning, Tim', 'four-seam'): "Cutter",
    ('Wilson, Tyler', 'sinker'): "Four-Seam",
    ('Wilson, Tyler', 'curveball'): "Slider",
    ('Bell, Brock', 'cutter'): "Four-Seam",
    ('Kelly, John', 'cutter'): "Four-Seam",
    ('Kelly, John', 'sinker'): "Four-Seam",
    ('Marshall, Dwayne', 'cutter'): "Four-Seam",    
    ('Scott, Brandon', 'sinker'): "Four-Seam",
    ('Scott, Brandon', 'slider'): "Sweeper",
    ('Scott, Brandon', 'curveball'): "Sweeper",
    ('Marshall, Dwayne', 'curveball'): "Slider",
    ('Lindgren, Jeff', 'sinker'): "Four-Seam",
    ('Crosby, Casey', 'curveball'): "Curveball",
    ('Timpanelli, Vin', 'sinker'): "Four-Seam",
    ('Veen, Zach', 'sinker'): "Four-Seam",
    ('Veen, Zach', 'curveball'): "Slider",
    ('Acosta, Jaykob', 'curveball'): "Slider",
    ('Acosta, Jaykob', 'cutter'): "Sinker",
    ('Acosta, Jaykob', 'four-seam'): "Sinker",
    ('Adams, Spencer', 'curveball'): "Slider",
    ('Alexander, Nathan', 'sinker'): "Four-Seam",
    ('Alexander, Nathan', 'slider'): "Cutter",
    ('Evans, Demarcus', 'cutter'): "Four-Seam",
    ('Hull, Denson', 'sinker'): "Four-Seam",
    ('Hull, Denson', 'curveball'): "Slider",
    ('Erwin, Chris', 'sinker'): "Four-Seam",
    ('Erwin, Chris', 'curveball'): "Curveball",
    ('Diaz, Andres', 'curveball'): "Slider",
    ('Long, Peyton', 'slider'): "Cutter",
    ('Long, Peyton', 'curveball'): "Sweeper",
    ('Cavaco, Keoni', 'four-seam'): "Sinker",
    ('Dalen, Kody', 'curveball'): "Slider",
    ('Hasty, Charlie', 'sinker'): "Four-Seam",
    ('Richardson, Ryan', 'splitter'): "Changeup",
    ('Torgerson, Cade', 'sinker'): "Four-Seam",
    ('Torgerson, Cade', 'curveball'): "Slider",
    ('Tyrpa, Sam', 'curveball'): "Slider",
    ('Zimmerman, Ryan', 'curveball'): "Slider",
    ('Zimmerman, Ryan', 'sinker'): "Four-Seam",
    ('Rivera, Jerryell', 'slider'): "Sweeper",
    ('Ligtenberg, Reese', 'curveball'): "Slider",
    ('Ligtenberg, Reese', 'sinker'): "Four-Seam",
    ('Braithwaite, Trey', 'cutter'): "Four-Seam",
    ('Braithwaite, Trey', 'curveball'): "Slider",
    ('Lacey, Steven', 'sinker'): "Four-Seam",
    ('Lacey, Steven', 'curveball'): "Slider",
    ('Lin, Eric', 'cutter'): "Slider",
    ('Marshall, Dwayne', 'sinker'): "Four-Seam",
    ('Nedrow, Jack', 'slider'): "Cutter",
    ('Johnson, Jordan', 'slider'): "Cutter",
    ('Miller, Brady', 'curveball'): "Slider",
    ('Chalus Jr., Eric', 'sinker'): "Four-Seam",
    # add more overrides here as needed:
    # ('Pitcher Name', 'raw_tag_name'): 'DesiredTag',
}

for (pitcher, raw_tag), correct_tag in manual_overrides.items():
    mask = (
        (team_pitchers['Pitcher'] == pitcher) &
        (team_pitchers['TaggedPitchType_clean'] == raw_tag)
    )
    if pitcher == 'Puckett, Brady' and raw_tag == 'slider':
        mask &= team_pitchers['HorzBreak'] > -8
    if pitcher == 'Bartow, Frankie' and raw_tag == 'sinker':
        mask &= team_pitchers['InducedVertBreak'] > 12.5
    if pitcher == 'Dorminy, Thomas' and raw_tag == 'slider':
        mask &= team_pitchers['HorzBreak'] < 10.5    
    if pitcher == 'Garcia, Julian' and raw_tag == 'curveball':
        mask &= team_pitchers['InducedVertBreak'] >  -10.5
    if pitcher == 'Goddard, Jackson' and raw_tag == 'curveball':
        mask &= team_pitchers['InducedVertBreak'] >  -10
    if pitcher == 'Hakanson, Jeff' and raw_tag == 'slider':
        mask &= team_pitchers['HorzBreak'] < -10.5
    if pitcher == 'Hendrickson, Josh' and raw_tag == 'slider':
        mask &= team_pitchers['HorzBreak'] < 2
    if pitcher == 'Martinez, Daniel' and raw_tag == 'slider':
        mask &= team_pitchers['RelSpeed'] > 85.9
    if pitcher == 'McKay, Tyler' and raw_tag == 'slider':
        mask &= team_pitchers['RelSpeed'] > 85.9
    if pitcher == 'Blake, Johnny' and raw_tag == 'changeup':
        mask &= team_pitchers['RelSpeed'] > 84.5
    if pitcher == 'Loukinen, Greg' and raw_tag == 'changeup':
        mask &= team_pitchers['RelSpeed'] > 84.5
    if pitcher == 'Bourassa, Landen' and raw_tag == 'slider':
        mask &= team_pitchers['HorzBreak'] > -5
    if pitcher == 'Cherry, Derrick' and raw_tag == 'slider':
        mask &= team_pitchers['RelSpeed'] > 86
    if pitcher == 'Echevarria, Juan' and raw_tag == 'changeup':
        mask &= team_pitchers['RelSpeed'] > 85
    if pitcher == 'Galindo, Jesse' and raw_tag == 'changeup':
        mask &= team_pitchers['RelSpeed'] > 81.5
    if pitcher == 'Lambson, Mitchell' and raw_tag == 'changeup':
        mask &= team_pitchers['RelSpeed'] > 77
    if pitcher == 'Onyshko, Ben' and raw_tag == 'changeup':
        mask &= team_pitchers['RelSpeed'] > 85
    if pitcher == 'Onyshko, Ben' and raw_tag == 'curveball':
        mask &= team_pitchers['HorzBreak'] < 10
    if pitcher == 'Yakel, Ryder' and raw_tag == 'curveball':
        mask &= team_pitchers['InducedVertBreak'] > -10
    if pitcher == 'Galindo, Jesse' and raw_tag == 'four-seam':
        mask &= team_pitchers['InducedVertBreak'] < 11
    if pitcher == 'Bourassa, Landen' and raw_tag == 'sinker':
        mask &= team_pitchers['RelSpeed'] < 85 
    team_pitchers.loc[mask, 'TaggedPitchType'] = correct_tag
# ——————————————————————————————————————————————
print(f"Overriding {mask.sum()} rows for {pitcher}:{raw_tag}")

mask_cepeda_four = (
    (team_pitchers['Pitcher'] == 'Cepeda, Felix') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto‐retagging {mask_cepeda_four.sum()} Cepeda sinkers (IVB > 10) → Four‐Seam")
team_pitchers.loc[mask_cepeda_four, 'TaggedPitchType'] = 'Four-Seam'
mask_gercken_drop = (
    (team_pitchers['Pitcher'] == 'Gercken, Nate') &
    (team_pitchers['HorzBreak'] < -14)
)
print(f"Dropping {mask_gercken_drop.sum()} Gercken pitches (HorzBreak < -14) from report")
team_pitchers = team_pitchers.loc[~mask_gercken_drop]

# —— Nate Gercken: force‐tag Four‐Seam with HorzBreak > 10 → Sinker ——
mask_gercken_sinker = (
    (team_pitchers['Pitcher'] == 'Gercken, Nate') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Auto‐retagging {mask_gercken_sinker.sum()} Gercken four-seams (HorzBreak > 10) → Sinker")
team_pitchers.loc[mask_gercken_sinker, 'TaggedPitchType'] = 'Sinker'
mask_gercken_drop2 = (
    (team_pitchers['Pitcher'] == 'Gercken, Nate') &
    (team_pitchers['InducedVertBreak'] < 0) &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Dropping {mask_gercken_drop2.sum()} Gercken pitches (IVB < 0 & HorzBreak > 10) from report")
team_pitchers = team_pitchers.loc[~mask_gercken_drop2]

mask_odonnell_four = (
    (team_pitchers['Pitcher'] == 'O\'Donnell, Brendan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto-retagging {mask_odonnell_four.sum()} O’Donnell changeups (IVB > 10) → Four-Seam")
team_pitchers.loc[mask_odonnell_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Brendan O’Donnell: force-tag Changeups with IVB < 10 → Sinker ——
mask_odonnell_sinker = (
    (team_pitchers['Pitcher'] == 'O\'Donnell, Brendan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['InducedVertBreak'] < 10)
)
print(f"Auto-retagging {mask_odonnell_sinker.sum()} O’Donnell changeups (IVB < 10) → Sinker")
team_pitchers.loc[mask_odonnell_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Brendan O’Donnell: force-tag Four-Seams with IVB < 5 → Sinker ——
mask_odonnell_four_to_sinker = (
    (team_pitchers['Pitcher'] == 'O\'Donnell, Brendan') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] < 5)
)
print(f"Auto-retagging {mask_odonnell_four_to_sinker.sum()} O’Donnell four-seams (IVB < 5) → Sinker")
team_pitchers.loc[mask_odonnell_four_to_sinker, 'TaggedPitchType'] = 'Sinker'
mask_scholten_cutter = (
    (team_pitchers['Pitcher'] == 'Scholten, JD') &
    (team_pitchers['HorzBreak'] >= -3) &
    (team_pitchers['HorzBreak'] <= 10)
)
print(f"Auto‐retagging {mask_scholten_cutter.sum()} Scholten pitches (–3 ≤ HorzBreak ≤ 10) → Cutter")
team_pitchers.loc[mask_scholten_cutter, 'TaggedPitchType'] = 'Cutter'
mask_drury_cutter = (
    (team_pitchers['Pitcher'] == 'Drury, Austin') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > -5) &
    (team_pitchers['HorzBreak'] < 7)
)
print(f"Auto‐retagging {mask_drury_cutter.sum()} Drury sliders (–5 < HorzBreak < 7) → Cutter")
team_pitchers.loc[mask_drury_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Austin Drury: force‐tag any pitch with HorzBreak > 8.5 → Sweeper ——
mask_drury_sweeper = (
    (team_pitchers['Pitcher'] == 'Drury, Austin') &
    (team_pitchers['HorzBreak'] > 8.5)
)
print(f"Auto‐retagging {mask_drury_sweeper.sum()} Drury pitches (HorzBreak > 8.5) → Sweeper")
team_pitchers.loc[mask_drury_sweeper, 'TaggedPitchType'] = 'Sweeper'

# —— Austin Drury: force‐tag Sinkers with IVB > 16 → Four‐Seam ——
mask_drury_four = (
    (team_pitchers['Pitcher'] == 'Drury, Austin') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 16)
)
print(f"Auto‐retagging {mask_drury_four.sum()} Drury sinkers (IVB > 16) → Four‐Seam")
team_pitchers.loc[mask_drury_four, 'TaggedPitchType'] = 'Four-Seam'
mask_guerrero_changeup = (
    (team_pitchers['Pitcher'] == 'Guerrero, Alberto') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] < 10)
)
print(f"Auto‐retagging {mask_guerrero_changeup.sum()} Guerrero four‐seams (IVB < 10) → Changeup")
team_pitchers.loc[mask_guerrero_changeup, 'TaggedPitchType'] = 'Changeup'

# —— Alberto Guerrero: force‐tag Four‐Seams with IVB > 10 → Sinker ——
mask_guerrero_sinker = (
    (team_pitchers['Pitcher'] == 'Guerrero, Alberto') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto‐retagging {mask_guerrero_sinker.sum()} Guerrero four‐seams (IVB > 10) → Sinker")
team_pitchers.loc[mask_guerrero_sinker, 'TaggedPitchType'] = 'Sinker'
mask_jandron_sinker = (
    (team_pitchers['Pitcher'] == 'Jandron, Tyler') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 84.5)
)
print(f"Auto‐retagging {mask_jandron_sinker.sum()} Jandron changeups (RelSpeed > 84.5) → Sinker")
team_pitchers.loc[mask_jandron_sinker, 'TaggedPitchType'] = 'Sinker'
mask_kiser_sinker = (
    (team_pitchers['Pitcher'] == 'Kiser, Kolby') &
    (team_pitchers['HorzBreak'] > 8) &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto‐retagging {mask_kiser_sinker.sum()} Kiser pitches (HorzBreak > 8 & RelSpeed > 86) → Sinker")
team_pitchers.loc[mask_kiser_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Kolby Kiser: force‐tag IVB > 0 & –5 ≤ HorzBreak ≤ 7 → Cutter ——
mask_kiser_cutter = (
    (team_pitchers['Pitcher'] == 'Kiser, Kolby') &
    (team_pitchers['InducedVertBreak'] > 0) &
    (team_pitchers['HorzBreak'] >= -5) &
    (team_pitchers['HorzBreak'] <= 7)
)
print(f"Auto‐retagging {mask_kiser_cutter.sum()} Kiser pitches (IVB > 0 & –5 ≤ HorzBreak ≤ 7) → Cutter")
team_pitchers.loc[mask_kiser_cutter, 'TaggedPitchType'] = 'Cutter'
mask_thiels_curve = (
    (team_pitchers['Pitcher'] == 'Thiels, Brenton') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 2)
)
print(f"Auto‐retagging {mask_thiels_curve.sum()} Thiels sliders (IVB < 2) → Curveball")
team_pitchers.loc[mask_thiels_curve, 'TaggedPitchType'] = 'Curveball'

# 2) IVB ≥ 2 → Cutter
mask_thiels_cutter = (
    (team_pitchers['Pitcher'] == 'Thiels, Brenton') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] >= 2)
)
print(f"Auto‐retagging {mask_thiels_cutter.sum()} Thiels sliders (IVB ≥ 2) → Cutter")
team_pitchers.loc[mask_thiels_cutter, 'TaggedPitchType'] = 'Cutter'
mask_jandron_sweeper = (
    (team_pitchers['Pitcher'] == 'Jandron, Tyler') &
    (team_pitchers['HorzBreak'] > 4)
)
print(f"Auto‐retagging {mask_jandron_sweeper.sum()} Jandron pitches (HorzBreak > 4) → Sweeper")
team_pitchers.loc[mask_jandron_sweeper, 'TaggedPitchType'] = 'Sweeper'

# 2) Curveballs with HorzBreak < 0 → Changeup
mask_jandron_curve = (
    (team_pitchers['Pitcher'] == 'Jandron, Tyler') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['HorzBreak'] < 0)
)
print(f"Auto‐retagging {mask_jandron_curve.sum()} Jandron curveballs (HorzBreak < 0) → Changeup")
team_pitchers.loc[mask_jandron_curve, 'TaggedPitchType'] = 'Changeup'

# 3) Sliders with HorzBreak < 3 → Cutter
mask_jandron_cutter = (
    (team_pitchers['Pitcher'] == 'Jandron, Tyler') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] < 3)
)
print(f"Auto‐retagging {mask_jandron_cutter.sum()} Jandron sliders (HorzBreak < 3) → Cutter")
team_pitchers.loc[mask_jandron_cutter, 'TaggedPitchType'] = 'Cutter'
mask_johnston_four = (
    (team_pitchers['Pitcher'] == 'Johnston, Kyle') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['HorzBreak'] < 12.5)
)
print(f"Auto-retagging {mask_johnston_four.sum()} Johnston sinkers (HorzBreak < 12.5) → Four-Seam")
team_pitchers.loc[mask_johnston_four, 'TaggedPitchType'] = 'Four-Seam'
mask_blain_cutter = (
    (team_pitchers['Pitcher'] == 'Blain, Nate') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 0) &
    (team_pitchers['HorzBreak'] > -12)
)
print(f"Auto‐retagging {mask_blain_cutter.sum()} Blain sliders (IVB > 0 & HorzBreak > -12) → Cutter")
team_pitchers.loc[mask_blain_cutter, 'TaggedPitchType'] = 'Cutter'
mask_blake_four = (
    (team_pitchers['Pitcher'] == 'Blake, Johnny') &
    (team_pitchers['TaggedPitchType'].isin(['Four-Seam', 'Sinker'])) &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto‐retagging {mask_blake_four.sum()} Blake four‐seams & sinkers (IVB > 10) → Four‐Seam")
team_pitchers.loc[mask_blake_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Johnny Blake: force‐tag Sliders with IVB > 0 & HorzBreak > -6 → Cutter ——
mask_blake_cutter = (
    (team_pitchers['Pitcher'] == 'Blake, Johnny') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 0) &
    (team_pitchers['HorzBreak'] > -6)
)
print(f"Auto‐retagging {mask_blake_cutter.sum()} Blake sliders (IVB > 0 & HorzBreak > -6) → Cutter")
team_pitchers.loc[mask_blake_cutter, 'TaggedPitchType'] = 'Cutter'
mask_cobos_sweeper = (
    (team_pitchers['Pitcher'] == 'Cobos, Franny') &
    (team_pitchers['HorzBreak'] < -10) &
    (team_pitchers['InducedVertBreak'] > -10)
)
print(f"Auto‐retagging {mask_cobos_sweeper.sum()} Cobos pitches (HorzBreak < -10 & IVB > -10) → Sweeper")
team_pitchers.loc[mask_cobos_sweeper, 'TaggedPitchType'] = 'Sweeper'
mask_castaneda_sinker = (
    (team_pitchers['Pitcher'] == 'Castaneda, Dylan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 86)
)
print(f"Auto‐retagging {mask_castaneda_sinker.sum()} Castaneda changeups (RelSpeed > 86) → Sinker")
team_pitchers.loc[mask_castaneda_sinker, 'TaggedPitchType'] = 'Sinker'
mask_mishoulam_sinker = (
    (team_pitchers['Pitcher'] == 'Mishoulam, Aaron') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Auto-retagging {mask_mishoulam_sinker.sum()} Mishoulam changeups (HorzBreak > 10) → Sinker")
team_pitchers.loc[mask_mishoulam_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Aaron Mishoulam: force‐tag Changeups with HorzBreak < 10 → Slider ——
mask_mishoulam_slider = (
    (team_pitchers['Pitcher'] == 'Mishoulam, Aaron') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['HorzBreak'] < 10)
)
print(f"Auto-retagging {mask_mishoulam_slider.sum()} Mishoulam changeups (HorzBreak < 10) → Slider")
team_pitchers.loc[mask_mishoulam_slider, 'TaggedPitchType'] = 'Slider'
mask_purnell_sinker = (
    (team_pitchers['Pitcher'] == 'Purnell, Blake') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 5)
)
print(f"Auto-retagging {mask_purnell_sinker.sum()} Purnell four-seams (HorzBreak > 5) → Sinker")
team_pitchers.loc[mask_purnell_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Blake Purnell: force‐tag Four‐Seams with HorzBreak < 5 → Cutter ——
mask_purnell_cutter = (
    (team_pitchers['Pitcher'] == 'Purnell, Blake') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] < 5)
)
print(f"Auto-retagging {mask_purnell_cutter.sum()} Purnell four-seams (HorzBreak < 5) → Cutter")
team_pitchers.loc[mask_purnell_cutter, 'TaggedPitchType'] = 'Cutter'
mask_snow_sinker = (
    (team_pitchers['Pitcher'] == 'Snow, Logan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 82)
)
print(f"Auto‐retagging {mask_snow_sinker.sum()} Snow changeups (RelSpeed > 82) → Sinker")
team_pitchers.loc[mask_snow_sinker, 'TaggedPitchType'] = 'Sinker'
mask_walker_four = (
    (team_pitchers['Pitcher'] == 'Walker, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 87.25)
)
print(f"Auto-retagging {mask_walker_four.sum()} Walker changeups (RelSpeed > 87.25) → Four-Seam")
team_pitchers.loc[mask_walker_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Matt Walker: force-tag Sliders with IVB > 0 → Cutter ——
mask_walker_cutter = (
    (team_pitchers['Pitcher'] == 'Walker, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto-retagging {mask_walker_cutter.sum()} Walker sliders (IVB > 0) → Cutter")
team_pitchers.loc[mask_walker_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Matt Walker: force-tag Sliders with IVB < 0 → Curveball ——
mask_walker_curve = (
    (team_pitchers['Pitcher'] == 'Walker, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto-retagging {mask_walker_curve.sum()} Walker sliders (IVB < 0) → Curveball")
team_pitchers.loc[mask_walker_curve, 'TaggedPitchType'] = 'Curveball'
mask_henley_cutter = (
    (team_pitchers['Pitcher'] == 'Henley, Blair') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > -10)
)
print(f"Auto-retagging {mask_henley_cutter.sum()} Henley sliders (HorzBreak > -10) → Cutter")
team_pitchers.loc[mask_henley_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Blair Henley: force-tag any pitch with HorzBreak < -10 & IVB > -12 → Sweeper ——
mask_henley_sweeper = (
    (team_pitchers['Pitcher'] == 'Henley, Blair') &
    (team_pitchers['HorzBreak'] < -10) &
    (team_pitchers['InducedVertBreak'] > -12)
)
print(f"Dropping {mask_henley_sweeper.sum()} Henley pitches (HorzBreak < -10 & IVB > -12) → Sweeper")
team_pitchers.loc[mask_henley_sweeper, 'TaggedPitchType'] = 'Sweeper'

# —— Blair Henley: force-tag Changeups with RelSpeed > 86.5 → Sinker ——
mask_henley_change = (
    (team_pitchers['Pitcher'] == 'Henley, Blair') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 86.5)
)
print(f"Auto-retagging {mask_henley_change.sum()} Henley changeups (RelSpeed > 86.5) → Sinker")
team_pitchers.loc[mask_henley_change, 'TaggedPitchType'] = 'Sinker'

# —— Blair Henley: force-tag Sinkers with IVB > 8 → Four-Seam ——
mask_henley_four = (
    (team_pitchers['Pitcher'] == 'Henley, Blair') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 8)
)
print(f"Auto-retagging {mask_henley_four.sum()} Henley sinkers (IVB > 8) → Four-Seam")
team_pitchers.loc[mask_henley_four, 'TaggedPitchType'] = 'Four-Seam'
mask_broadway_cutter = (
    (team_pitchers['Pitcher'] == 'Broadway, Taylor') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto‐retagging {mask_broadway_cutter.sum()} Broadway sliders (IVB > 0) → Cutter")
team_pitchers.loc[mask_broadway_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Taylor Broadway: force‐tag Sliders with IVB < 0 → Curveball ——
mask_broadway_curve = (
    (team_pitchers['Pitcher'] == 'Broadway, Taylor') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto‐retagging {mask_broadway_curve.sum()} Broadway sliders (IVB < 0) → Curveball")
team_pitchers.loc[mask_broadway_curve, 'TaggedPitchType'] = 'Curveball'
mask_craft_changeup = (
    (team_pitchers['Pitcher'] == 'Craft, Derek') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto‐retagging {mask_craft_changeup.sum()} Craft sinkers (RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_craft_changeup, 'TaggedPitchType'] = 'Changeup'

# —— Derek Craft: force‐tag remaining Sinkers → Four-Seam ——
mask_craft_four = (
    (team_pitchers['Pitcher'] == 'Craft, Derek') &
    (team_pitchers['TaggedPitchType'] == 'Sinker')
)
print(f"Auto‐retagging {mask_craft_four.sum()} Craft sinkers → Four-Seam")
team_pitchers.loc[mask_craft_four, 'TaggedPitchType'] = 'Four-Seam'
mask_lopez_sinker = (
    (team_pitchers['Pitcher'] == 'Lopez, Cristian') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Auto-retagging {mask_lopez_sinker.sum()} Lopez four-seams (HorzBreak > 10) → Sinker")
team_pitchers.loc[mask_lopez_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Cristian Lopez: force‐tag Sliders with RelSpeed > 84.5 → Cutter ——
mask_lopez_cutter = (
    (team_pitchers['Pitcher'] == 'Lopez, Cristian') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 84.5)
)
print(f"Auto-retagging {mask_lopez_cutter.sum()} Lopez sliders (RelSpeed > 84.5) → Cutter")
team_pitchers.loc[mask_lopez_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Cristian Lopez: force‐tag all Curveballs → Slider ——
mask_lopez_curve = (
    (team_pitchers['Pitcher'] == 'Lopez, Cristian') &
    (team_pitchers['TaggedPitchType'] == 'Curveball')
)
print(f"Auto-retagging {mask_lopez_curve.sum()} Lopez curveballs → Slider")
team_pitchers.loc[mask_lopez_curve, 'TaggedPitchType'] = 'Slider'
mask_rivera_four = (
    (team_pitchers['Pitcher'] == 'Rivera, Jerryell') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['HorzBreak'] > -10)
)
print(f"Auto-retagging {mask_rivera_four.sum()} Rivera sinkers (HorzBreak > -10) → Four-Seam")
team_pitchers.loc[mask_rivera_four, 'TaggedPitchType'] = 'Four-Seam'
mask_shawaryn_four = (
    (team_pitchers['Pitcher'] == 'Shawaryn, Mike') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['HorzBreak'] < 18)
)
print(f"Auto‐retagging {mask_shawaryn_four.sum()} Shawaryn sinkers (HorzBreak < 18) → Four‐Seam")
team_pitchers.loc[mask_shawaryn_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Mike Shawaryn: force‐tag Changeups with RelSpeed > 87.5 → Four‐Seam ——
mask_shawaryn_change = (
    (team_pitchers['Pitcher'] == 'Shawaryn, Mike') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 87.5)
)
print(f"Auto‐retagging {mask_shawaryn_change.sum()} Shawaryn changeups (RelSpeed > 87.5) → Four‐Seam")
team_pitchers.loc[mask_shawaryn_change, 'TaggedPitchType'] = 'Four-Seam'

# —— Mike Shawaryn: force‐tag Sliders with RelSpeed > 83.5 → Cutter ——
mask_shawaryn_cutter = (
    (team_pitchers['Pitcher'] == 'Shawaryn, Mike') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 82)
)
print(f"Auto‐retagging {mask_shawaryn_cutter.sum()} Shawaryn sliders (RelSpeed > 83.5) → Cutter")
team_pitchers.loc[mask_shawaryn_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Mike Shawaryn: force‐tag Sliders with RelSpeed < 83.5 → Curveball ——
mask_shawaryn_curve = (
    (team_pitchers['Pitcher'] == 'Shawaryn, Mike') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] < 82)
)
print(f"Auto‐retagging {mask_shawaryn_curve.sum()} Shawaryn sliders (RelSpeed < 83.5) → Curveball")
team_pitchers.loc[mask_shawaryn_curve, 'TaggedPitchType'] = 'Curveball'
mask_wood_four = (
    (team_pitchers['Pitcher'] == 'Wood, Zeke') &
    (team_pitchers['InducedVertBreak'] > 13.5)
)
print(f"Auto‐retagging {mask_wood_four.sum()} Wood pitches (IVB > 13.5) → Four‐Seam")
team_pitchers.loc[mask_wood_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Zeke Wood: force‐tag Sliders with RelSpeed > 85 → Cutter ——
mask_wood_cutter = (
    (team_pitchers['Pitcher'] == 'Wood, Zeke') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto‐retagging {mask_wood_cutter.sum()} Wood sliders (RelSpeed > 85) → Cutter")
team_pitchers.loc[mask_wood_cutter, 'TaggedPitchType'] = 'Cutter'
mask_stevens_slider = (
    (team_pitchers['Pitcher'] == 'Stevens, Will') &
    (team_pitchers['RelSpeed'] < 85)
)
print(f"Auto‐retagging {mask_stevens_slider.sum()} Stevens pitches (RelSpeed < 85) → Slider")
team_pitchers.loc[mask_stevens_slider, 'TaggedPitchType'] = 'Slider'

# —— Will Stevens: force‐tag 0 ≤ IVB ≤ 11.5 → Cutter ——
mask_stevens_cutter = (
    (team_pitchers['Pitcher'] == 'Stevens, Will') &
    (team_pitchers['InducedVertBreak'] >= 0) &
    (team_pitchers['InducedVertBreak'] <= 11.5)
)
print(f"Auto‐retagging {mask_stevens_cutter.sum()} Stevens pitches (0 ≤ IVB ≤ 11.5) → Cutter")
team_pitchers.loc[mask_stevens_cutter, 'TaggedPitchType'] = 'Cutter'
mask_pilot_four = (
    (team_pitchers['Pitcher'] == 'Pilot, Kelvan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 87)
)
print(f"Auto‐retagging {mask_pilot_four.sum()} Pilot changeups (RelSpeed > 87) → Four‐Seam")
team_pitchers.loc[mask_pilot_four, 'TaggedPitchType'] = 'Four-Seam'
mask_mechals_sinker = (
    (team_pitchers['Pitcher'] == 'Mechals, Kade') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 84.5)
)
print(f"Auto‐retagging {mask_mechals_sinker.sum()} Mechals changeups (RelSpeed > 84.5) → Sinker")
team_pitchers.loc[mask_mechals_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Kade Mechals: force‐tag Sliders with RelSpeed > 83 → Cutter ——
mask_mechals_cutter = (
    (team_pitchers['Pitcher'] == 'Mechals, Kade') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 83)
)
print(f"Auto‐retagging {mask_mechals_cutter.sum()} Mechals sliders (RelSpeed > 83) → Cutter")
team_pitchers.loc[mask_mechals_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Kade Mechals: force‐tag all Curveballs → Slider ——
mask_mechals_curve = (
    (team_pitchers['Pitcher'] == 'Mechals, Kade') &
    (team_pitchers['TaggedPitchType'] == 'Curveball')
)
print(f"Auto‐retagging {mask_mechals_curve.sum()} Mechals curveballs → Slider")
team_pitchers.loc[mask_mechals_curve, 'TaggedPitchType'] = 'Slider'

# —— Kade Mechals: force‐tag Sinkers with IVB > 12 → Four‐Seam ——
mask_mechals_four = (
    (team_pitchers['Pitcher'] == 'Mechals, Kade') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 12)
)
print(f"Auto‐retagging {mask_mechals_four.sum()} Mechals sinkers (IVB > 12) → Four‐Seam")
team_pitchers.loc[mask_mechals_four, 'TaggedPitchType'] = 'Four-Seam'

mask_davidson_slider = (
    (team_pitchers['Pitcher'] == 'Davidson, Zach') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto-retagging {mask_davidson_slider.sum()} Davidson changeups (IVB < 0) → Slider")
team_pitchers.loc[mask_davidson_slider, 'TaggedPitchType'] = 'Slider'

# —— Zach Davidson: force-tag Four-Seams with RelSpeed < 88 → Changeup ——
mask_davidson_changeup = (
    (team_pitchers['Pitcher'] == 'Davidson, Zach') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto-retagging {mask_davidson_changeup.sum()} Davidson four-seams (RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_davidson_changeup, 'TaggedPitchType'] = 'Changeup'
mask_bell_changeup = (
    (team_pitchers['Pitcher'] == 'Bell, Brock') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['RelSpeed'] < 86.5)
)
print(f"Auto-retagging {mask_bell_changeup.sum()} Bell four-seams (RelSpeed < 86.5) → Changeup")
team_pitchers.loc[mask_bell_changeup, 'TaggedPitchType'] = 'Changeup'

# —— Brock Bell: force-tag Sliders with RelSpeed < 78.5 → Curveball ——
mask_bell_curve = (
    (team_pitchers['Pitcher'] == 'Bell, Brock') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] < 78.5)
)
print(f"Auto-retagging {mask_bell_curve.sum()} Bell sliders (RelSpeed < 78.5) → Curveball")
team_pitchers.loc[mask_bell_curve, 'TaggedPitchType'] = 'Curveball'

mask_lin_drop = (
    (team_pitchers['Pitcher'] == 'Lin, Eric') &
    (team_pitchers['TaggedPitchType'] == 'Sinker')
)
print(f"Dropping {mask_lin_drop.sum()} Lin sinkers from report")
team_pitchers = team_pitchers.loc[~mask_lin_drop]

mask_bell_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Bell, Brock') &
    (team_pitchers['TaggedPitchType'] == 'Slider')
)
print(f"Auto‐retagging {mask_bell_slider_to_cutter.sum()} Bell sliders → Cutter")
team_pitchers.loc[mask_bell_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Brock Bell: force‐tag Curveballs with RelSpeed > 80 → Cutter ——
mask_bell_curve_to_cutter = (
    (team_pitchers['Pitcher'] == 'Bell, Brock') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['RelSpeed'] > 80)
)
print(f"Auto‐retagging {mask_bell_curve_to_cutter.sum()} Bell curveballs (RelSpeed > 80) → Cutter")
team_pitchers.loc[mask_bell_curve_to_cutter, 'TaggedPitchType'] = 'Cutter'

mask_keys_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Keys, JC') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 5)
)
print(f"Auto-retagging {mask_keys_slider_to_cutter.sum()} Keys sliders (IVB > 5) → Cutter")
team_pitchers.loc[mask_keys_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— JC Keys: force-tag Four-Seams with IVB < 12 & HorzBreak < 0 → Cutter ——
mask_keys_four_to_cutter = (
    (team_pitchers['Pitcher'] == 'Keys, JC') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] < 12) &
    (team_pitchers['HorzBreak'] < 0)
)
print(f"Auto-retagging {mask_keys_four_to_cutter.sum()} Keys four-seams (IVB < 12 & HorzBreak < 0) → Cutter")
team_pitchers.loc[mask_keys_four_to_cutter, 'TaggedPitchType'] = 'Cutter'
mask_keys_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Keys, JC') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['HorzBreak'] > -10) &
    (team_pitchers['InducedVertBreak'] > -10)
)
print(f"Auto‐retagging {mask_keys_curve_to_slider.sum()} Keys curveballs (HorzBreak > -10 & IVB > -10) → Slider")
team_pitchers.loc[mask_keys_curve_to_slider, 'TaggedPitchType'] = 'Slider'
mask_nedrow_four = (
    (team_pitchers['Pitcher'] == 'Nedrow, Jack') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['InducedVertBreak'] > 16)
)
print(f"Auto‐retagging {mask_nedrow_four.sum()} Nedrow cutters (IVB > 16) → Four‐Seam")
team_pitchers.loc[mask_nedrow_four, 'TaggedPitchType'] = 'Four-Seam'

# 2) Cutters with IVB < 16 → Slider
mask_nedrow_slider = (
    (team_pitchers['Pitcher'] == 'Nedrow, Jack') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['InducedVertBreak'] < 16)
)
print(f"Auto‐retagging {mask_nedrow_slider.sum()} Nedrow cutters (IVB < 16) → Slider")
team_pitchers.loc[mask_nedrow_slider, 'TaggedPitchType'] = 'Cutter'

# —— Jack Nedrow: force‐tag Sliders with IVB < 0 → Curveball ——
mask_nedrow_curve = (
    (team_pitchers['Pitcher'] == 'Nedrow, Jack') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto‐retagging {mask_nedrow_curve.sum()} Nedrow sliders (IVB < 0) → Curveball")
team_pitchers.loc[mask_nedrow_curve, 'TaggedPitchType'] = 'Curveball'
mask_lindgren_changeup = (
    (team_pitchers['Pitcher'] == 'Lindgren, Jeff') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto-retagging {mask_lindgren_changeup.sum()} Lindgren sinkers (RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_lindgren_changeup, 'TaggedPitchType'] = 'Changeup'

# —— Jeff Lindgren: force‐tag Sliders with RelSpeed > 83.5 → Cutter ——
mask_lindgren_cutter = (
    (team_pitchers['Pitcher'] == 'Lindgren, Jeff') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 83.5)
)
print(f"Auto-retagging {mask_lindgren_cutter.sum()} Lindgren sliders (RelSpeed > 83.5) → Cutter")
team_pitchers.loc[mask_lindgren_cutter, 'TaggedPitchType'] = 'Cutter'

mask_lindgren_four_to_change = (
    (team_pitchers['Pitcher'] == 'Lindgren, Jeff') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto-retagging {mask_lindgren_four_to_change.sum()} Lindgren four-seams (RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_lindgren_four_to_change, 'TaggedPitchType'] = 'Changeup'

mask_mazza_sinker = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto‐retagging {mask_mazza_sinker.sum()} Mazza changeups (RelSpeed > 85) → Sinker")
team_pitchers.loc[mask_mazza_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Chris Mazza: force‐tag all Splitters → Changeup ——
mask_mazza_splitter = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Splitter')
)
print(f"Auto‐retagging {mask_mazza_splitter.sum()} Mazza splitters → Changeup")
team_pitchers.loc[mask_mazza_splitter, 'TaggedPitchType'] = 'Changeup'

# —— Chris Mazza: force‐tag all Curveballs → Sweeper ——
mask_mazza_curve = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Curveball')
)
print(f"Auto‐retagging {mask_mazza_curve.sum()} Mazza curveballs → Sweeper")
team_pitchers.loc[mask_mazza_curve, 'TaggedPitchType'] = 'Sweeper'

# —— Chris Mazza: force‐tag Sliders with RelSpeed > 82 → Cutter ——
mask_mazza_cutter = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 82)
)
print(f"Auto‐retagging {mask_mazza_cutter.sum()} Mazza sliders (RelSpeed > 82) → Cutter")
team_pitchers.loc[mask_mazza_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Chris Mazza: force‐tag Sliders with RelSpeed ≤ 82 → Sweeper ——
mask_mazza_sweeper_slider = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] <= 82)
)
print(f"Auto‐retagging {mask_mazza_sweeper_slider.sum()} Mazza sliders (RelSpeed ≤ 82) → Sweeper")
team_pitchers.loc[mask_mazza_sweeper_slider, 'TaggedPitchType'] = 'Sweeper'
mask_mazza_four_to_sinker = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] < 18)
)
print(f"Auto‐retagging {mask_mazza_four_to_sinker.sum()} Mazza four-seams (IVB < 18) → Sinker")
team_pitchers.loc[mask_mazza_four_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Chris Mazza: drop Four-Seams with IVB > 18 ——
mask_mazza_drop_four = (
    (team_pitchers['Pitcher'] == 'Mazza, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] > 18)
)
print(f"Dropping {mask_mazza_drop_four.sum()} Mazza four-seams (IVB > 18) from report")
team_pitchers = team_pitchers.loc[~mask_mazza_drop_four]

mask_muir_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Muir, Westin') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > -8.5)
)
print(f"Auto‐retagging {mask_muir_curve_to_slider.sum()} Muir curveballs (IVB > -8.5) → Slider")
team_pitchers.loc[mask_muir_curve_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Westin Muir: force‐tag Changeups with IVB > 10 → Four‐Seam ——
mask_muir_change_to_four = (
    (team_pitchers['Pitcher'] == 'Muir, Westin') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto‐retagging {mask_muir_change_to_four.sum()} Muir changeups (IVB > 10) → Four‐Seam")
team_pitchers.loc[mask_muir_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Westin Muir: force‐tag Changeups with IVB < -2 → Curveball ——
mask_muir_change_to_curve = (
    (team_pitchers['Pitcher'] == 'Muir, Westin') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['InducedVertBreak'] < -2)
)
print(f"Auto‐retagging {mask_muir_change_to_curve.sum()} Muir changeups (IVB < -2) → Curveball")
team_pitchers.loc[mask_muir_change_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— Westin Muir: force‐tag Sliders with IVB > 10 → Changeup ——
mask_muir_slider_to_change = (
    (team_pitchers['Pitcher'] == 'Muir, Westin') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto‐retagging {mask_muir_slider_to_change.sum()} Muir sliders (IVB > 10) → Changeup")
team_pitchers.loc[mask_muir_slider_to_change, 'TaggedPitchType'] = 'Four-Seam'

mask_ash_slider_to_change = (
    (team_pitchers['Pitcher'] == 'Ash, Konnor') &
    (team_pitchers['TaggedPitchType'] == 'Slider')
)
print(f"Auto-retagging {mask_ash_slider_to_change.sum()} Ash sliders → Changeup")
team_pitchers.loc[mask_ash_slider_to_change, 'TaggedPitchType'] = 'Changeup'

# —— Konnor Ash: force‐tag Cutters with IVB > 12.5 → Four‐Seam ——
mask_ash_cutter_to_four = (
    (team_pitchers['Pitcher'] == 'Ash, Konnor') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['InducedVertBreak'] > 12.5)
)
print(f"Auto-retagging {mask_ash_cutter_to_four.sum()} Ash cutters (IVB > 12.5) → Four‐Seam")
team_pitchers.loc[mask_ash_cutter_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Konnor Ash: force‐tag Cutters with IVB ≤ 12.5 → Changeup ——
mask_ash_cutter_to_change = (
    (team_pitchers['Pitcher'] == 'Ash, Konnor') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['InducedVertBreak'] <= 12.5)
)
print(f"Auto-retagging {mask_ash_cutter_to_change.sum()} Ash cutters (IVB ≤ 12.5) → Changeup")
team_pitchers.loc[mask_ash_cutter_to_change, 'TaggedPitchType'] = 'Changeup'

# —— Konnor Ash: force‐tag Changeups with IVB < -2 → Curveball ——
mask_ash_change_to_curve = (
    (team_pitchers['Pitcher'] == 'Ash, Konnor') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['InducedVertBreak'] < -2)
)
print(f"Auto-retagging {mask_ash_change_to_curve.sum()} Ash changeups (IVB < -2) → Curveball")
team_pitchers.loc[mask_ash_change_to_curve, 'TaggedPitchType'] = 'Curveball'
mask_nissen_curve = (
    (team_pitchers['Pitcher'] == 'Nissen, Logan') &
    (team_pitchers['TaggedPitchType'] == 'Curveball')
)
print(f"Auto‐retagging {mask_nissen_curve.sum()} Nissen curveballs → Slider")
team_pitchers.loc[mask_nissen_curve, 'TaggedPitchType'] = 'Slider'

# —— Logan Nissen: force‐tag Sinkers with IVB > 11 → Four‐Seam ——
mask_nissen_sinker_four = (
    (team_pitchers['Pitcher'] == 'Nissen, Logan') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 11)
)
print(f"Auto‐retagging {mask_nissen_sinker_four.sum()} Nissen sinkers (IVB > 11) → Four‐Seam")
team_pitchers.loc[mask_nissen_sinker_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Logan Nissen: drop all other Sinkers ——
mask_nissen_sinker_drop = (
    (team_pitchers['Pitcher'] == 'Nissen, Logan') &
    (team_pitchers['TaggedPitchType'] == 'Sinker')
    # at this point only those with IVB ≤ 11 remain
)
print(f"Dropping {mask_nissen_sinker_drop.sum()} Nissen sinkers (IVB ≤ 11) from report")
team_pitchers = team_pitchers.loc[~mask_nissen_sinker_drop]
mask_nissen_slider_four = (
    (team_pitchers['Pitcher'] == 'Nissen, Logan') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 10)
)
print(f"Auto‐retagging {mask_nissen_slider_four.sum()} Nissen sliders (IVB > 10) → Four‐Seam")
team_pitchers.loc[mask_nissen_slider_four, 'TaggedPitchType'] = 'Four-Seam'

mask_crosby_curve = (
    (team_pitchers['Pitcher'] == 'Crosby, Casey') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < -4.5)
)
print(f"Auto‐retagging {mask_crosby_curve.sum()} Crosby sliders (IVB < 4.5) → Curveball")
team_pitchers.loc[mask_crosby_curve, 'TaggedPitchType'] = 'Curveball'

mask_stevenson_four = (
    (team_pitchers['Pitcher'] == 'Stevenson, Jake') &
    (team_pitchers['TaggedPitchType'] == 'Sinker')
)
print(f"Auto‐retagging {mask_stevenson_four.sum()} Stevenson sinkers → Four‐Seam")
team_pitchers.loc[mask_stevenson_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Jake Stevenson: force‐tag Sliders with RelSpeed > 85 → Cutter ——
mask_stevenson_cutter = (
    (team_pitchers['Pitcher'] == 'Stevenson, Jake') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto‐retagging {mask_stevenson_cutter.sum()} Stevenson sliders (RelSpeed > 85) → Cutter")
team_pitchers.loc[mask_stevenson_cutter, 'TaggedPitchType'] = 'Cutter'

mask_martinson_change_to_four = (
    (team_pitchers['Pitcher'] == 'Martinson, Jordan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup')
)
print(f"Auto‐retagging {mask_martinson_change_to_four.sum()} Martinson changeups → Four‐Seam")
team_pitchers.loc[mask_martinson_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Jordan Martinson: force‐tag Cutters with IVB > 8.5 → Four‐Seam ——
mask_martinson_cutter_to_four = (
    (team_pitchers['Pitcher'] == 'Martinson, Jordan') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['InducedVertBreak'] > 8.5)
)
print(f"Auto‐retagging {mask_martinson_cutter_to_four.sum()} Martinson cutters (IVB > 8.5) → Four‐Seam")
team_pitchers.loc[mask_martinson_cutter_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Jordan Martinson: force‐tag Sliders with IVB > 10 & HorzBreak < 3.5 → Four‐Seam ——
mask_martinson_slider_shape_to_four = (
    (team_pitchers['Pitcher'] == 'Martinson, Jordan') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 10) &
    (team_pitchers['HorzBreak'] < 3.5)
)
print(f"Auto‐retagging {mask_martinson_slider_shape_to_four.sum()} Martinson sliders (IVB > 10 & HorzBreak < 3.5) → Four‐Seam")
team_pitchers.loc[mask_martinson_slider_shape_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Jordan Martinson: force‐tag Sliders with RelSpeed > 83.5 → Cutter ——
mask_martinson_slider_speed_to_cutter = (
    (team_pitchers['Pitcher'] == 'Martinson, Jordan') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 83.5)
)
print(f"Auto‐retagging {mask_martinson_slider_speed_to_cutter.sum()} Martinson sliders (RelSpeed > 83.5) → Cutter")
team_pitchers.loc[mask_martinson_slider_speed_to_cutter, 'TaggedPitchType'] = 'Cutter'
mask_martinson_curve_hb_to_slider = (
    (team_pitchers['Pitcher'] == 'Martinson, Jordan') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['HorzBreak'] < 12)
)
print(f"Auto-retagging {mask_martinson_curve_hb_to_slider.sum()} Martinson curveballs (HorzBreak < 12) → Slider")
team_pitchers.loc[mask_martinson_curve_hb_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Jordan Martinson: force‐tag Sliders with HorzBreak > 15 → Sweeper ——
mask_martinson_slider_hb_to_sweeper = (
    (team_pitchers['Pitcher'] == 'Martinson, Jordan') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > 15)
)
print(f"Auto-retagging {mask_martinson_slider_hb_to_sweeper.sum()} Martinson sliders (HorzBreak > 15) → Sweeper")
team_pitchers.loc[mask_martinson_slider_hb_to_sweeper, 'TaggedPitchType'] = 'Sweeper'

mask_fox_change_to_four = (
    (team_pitchers['Pitcher'] == 'Fox, Jack') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['SpinRate'] > 1600)
)
print(f"Auto‐retagging {mask_fox_change_to_four.sum()} Fox changeups (SpinRate > 1600) → Four‐Seam")
team_pitchers.loc[mask_fox_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Jack Fox: force‐tag Sliders with RelSpeed > 85.5 → Four‐Seam ——
mask_fox_slider_to_four = (
    (team_pitchers['Pitcher'] == 'Fox, Jack') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 85.5)
)
print(f"Auto‐retagging {mask_fox_slider_to_four.sum()} Fox sliders (RelSpeed > 85.5) → Four‐Seam")
team_pitchers.loc[mask_fox_slider_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Jack Fox: force‐tag Curveballs with IVB > -2 → Slider ——
mask_fox_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Fox, Jack') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > -2.5)
)
print(f"Auto‐retagging {mask_fox_curve_to_slider.sum()} Fox curveballs (IVB > -2) → Slider")
team_pitchers.loc[mask_fox_curve_to_slider, 'TaggedPitchType'] = 'Slider'

mask_gudaitis_hb_cutter = (
    (team_pitchers['Pitcher'] == 'Gudaitis, Quinn') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > -9.75)
)
print(f"Auto‐retagging {mask_gudaitis_hb_cutter.sum()} Gudaitis sliders (HorzBreak > -7) → Cutter")
team_pitchers.loc[mask_gudaitis_hb_cutter, 'TaggedPitchType'] = 'Cutter'

mask_timpanelli_four = (
    (team_pitchers['Pitcher'] == 'Timpanelli, Vin') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 86)
)
print(f"Auto‐retagging {mask_timpanelli_four.sum()} Timpanelli changeups (RelSpeed > 86) → Four‐Seam")
team_pitchers.loc[mask_timpanelli_four, 'TaggedPitchType'] = 'Four-Seam'

mask_veen_change_to_four = (
    (team_pitchers['Pitcher'] == 'Veen, Zach') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 84)
)
print(f"Auto‐retagging {mask_veen_change_to_four.sum()} Veen changeups (RelSpeed > 84) → Four‐Seam")
team_pitchers.loc[mask_veen_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Zach Veen: drop Four‐Seams with HorzBreak > 0 ——
mask_veen_drop_four = (
    (team_pitchers['Pitcher'] == 'Veen, Zach') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 0)
)
print(f"Dropping {mask_veen_drop_four.sum()} Veen four‐seams (HorzBreak > 0) from report")
team_pitchers = team_pitchers.loc[~mask_veen_drop_four]

mask_coats_drop_four = (
    (team_pitchers['Pitcher'] == 'Coats, Jacob') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam')
)
print(f"Dropping {mask_coats_drop_four.sum()} Coats four-seams from report")
team_pitchers = team_pitchers.loc[~mask_coats_drop_four]

mask_acosta_change85 = (
    (team_pitchers['Pitcher'] == 'Acosta, Jaykob') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto-retagging {mask_acosta_change85.sum()} Acosta changeups (RelSpeed > 85) → Sinker")
team_pitchers.loc[mask_acosta_change85, 'TaggedPitchType'] = 'Sinker'

# —— Jaykob Acosta: force-tag Sinkers with HorzBreak < 10 → Four-Seam ——
mask_acosta_sinker_to_four = (
    (team_pitchers['Pitcher'] == 'Acosta, Jaykob') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['HorzBreak'] < 10)
)
print(f"Auto-retagging {mask_acosta_sinker_to_four.sum()} Acosta sinkers (HorzBreak < 10) → Four-Seam")
team_pitchers.loc[mask_acosta_sinker_to_four, 'TaggedPitchType'] = 'Sinker'

# —— Jaykob Acosta: force-tag Sliders with HorzBreak > 0 → Four-Seam ——
mask_acosta_slider_to_four = (
    (team_pitchers['Pitcher'] == 'Acosta, Jaykob') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > 0)
)
print(f"Auto-retagging {mask_acosta_slider_to_four.sum()} Acosta sliders (HorzBreak > 0) → Four-Seam")
team_pitchers.loc[mask_acosta_slider_to_four, 'TaggedPitchType'] = 'Sinker'

mask_adams_changeup = (
    (team_pitchers['Pitcher'] == 'Adams, Spencer') &
    (team_pitchers['TaggedPitchType'].isin(['Sinker', 'Four-Seam'])) &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto-retagging {mask_adams_changeup.sum()} Adams pitches (Sinker/Four-Seam & RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_adams_changeup, 'TaggedPitchType'] = 'Changeup'

mask_evans_cutter = (
    (team_pitchers['Pitcher'] == 'Evans, Demarcus') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 4)
)
print(f"Auto‐retagging {mask_evans_cutter.sum()} Evans sliders (IVB > 4) → Cutter")
team_pitchers.loc[mask_evans_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Demarcus Evans: force‐tag Sliders with IVB < 4 → Curveball ——
mask_evans_curve = (
    (team_pitchers['Pitcher'] == 'Evans, Demarcus') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 4)
)
print(f"Auto‐retagging {mask_evans_curve.sum()} Evans sliders (IVB < 4) → Curveball")
team_pitchers.loc[mask_evans_curve, 'TaggedPitchType'] = 'Curveball'
mask_lane_cut_to_four = (
    (team_pitchers['Pitcher'] == 'Lane, Dawson') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['RelSpeed'] > 86.5)
)
print(f"Auto‐retagging {mask_lane_cut_to_four.sum()} Lane cutters (RelSpeed > 86.5) → Four‐Seam")
team_pitchers.loc[mask_lane_cut_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Dawson Lane: force‐tag Cutters with RelSpeed ≤ 86.5 → Slider ——
mask_lane_cut_to_slider = (
    (team_pitchers['Pitcher'] == 'Lane, Dawson') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['RelSpeed'] <= 86.5)
)
print(f"Auto‐retagging {mask_lane_cut_to_slider.sum()} Lane cutters (RelSpeed ≤ 86.5) → Slider")
team_pitchers.loc[mask_lane_cut_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Dawson Lane: force‐tag Sliders with HorzBreak > 0 & IVB < 0 → Curveball ——
mask_lane_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Lane, Dawson') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > 0) &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto‐retagging {mask_lane_slider_to_curve.sum()} Lane sliders (HorzBreak > 0 & IVB < 0) → Curveball")
team_pitchers.loc[mask_lane_slider_to_curve, 'TaggedPitchType'] = 'Curveball'
mask_reed_sinker_to_four = (
    (team_pitchers['Pitcher'] == 'Reed, Cody') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['HorzBreak'] > -12)
)
print(f"Auto‐retagging {mask_reed_sinker_to_four.sum()} Reed sinkers (HorzBreak > -12) → Four‐Seam")
team_pitchers.loc[mask_reed_sinker_to_four, 'TaggedPitchType'] = 'Four-Seam'
mask_erwin_change_to_four = (
    (team_pitchers['Pitcher'] == 'Erwin, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 83)
)
print(f"Auto‐retagging {mask_erwin_change_to_four.sum()} Erwin changeups (RelSpeed > 83) → Four‐Seam")
team_pitchers.loc[mask_erwin_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Chris Erwin: force‐tag Curveballs with IVB > -8.5 → Slider ——
mask_erwin_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Erwin, Chris') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > -8.5)
)
print(f"Auto‐retagging {mask_erwin_curve_to_slider.sum()} Erwin curveballs (IVB > -8.5) → Slider")
team_pitchers.loc[mask_erwin_curve_to_slider, 'TaggedPitchType'] = 'Slider'

mask_diaz_sinker_to_four = (
    (team_pitchers['Pitcher'] == 'Diaz, Andres') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 10) &
    (team_pitchers['HorzBreak'] < 14)
)
print(f"Auto‐retagging {mask_diaz_sinker_to_four.sum()} Diaz sinkers (IVB > 10 & HorzBreak < 14) → Four-Seam")
team_pitchers.loc[mask_diaz_sinker_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Andres Diaz: force‐tag Changeups with RelSpeed > 86 → Four‐Seam ——
mask_diaz_change_to_four = (
    (team_pitchers['Pitcher'] == 'Diaz, Andres') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 86)
)
print(f"Auto‐retagging {mask_diaz_change_to_four.sum()} Diaz changeups (RelSpeed > 86) → Four-Seam")
team_pitchers.loc[mask_diaz_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

mask_long_sweeper = (
    (team_pitchers['Pitcher'] == 'Long, Peyton') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] < -10)
)
print(f"Auto‐retagging {mask_long_sweeper.sum()} Long sliders (HorzBreak < -10) → Sweeper")
team_pitchers.loc[mask_long_sweeper, 'TaggedPitchType'] = 'Sweeper'

# —— Peyton Long: force‐tag Sinkers with RelSpeed < 88 → Changeup ——
mask_long_changeup = (
    (team_pitchers['Pitcher'] == 'Long, Peyton') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto‐retagging {mask_long_changeup.sum()} Long sinkers (RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_long_changeup, 'TaggedPitchType'] = 'Changeup'

mask_zaragoza_cutter_to_four = (
    (team_pitchers['Pitcher'] == 'Zaragoza, Ernesto') &
    (team_pitchers['TaggedPitchType'] == 'Cutter')
)
print(f"Auto‐retagging {mask_zaragoza_cutter_to_four.sum()} Zaragoza cutters → Four‐Seam")
team_pitchers.loc[mask_zaragoza_cutter_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Ernesto Zaragoza: force‐tag Four‐Seams with HorzBreak > 10 → Sinker ——
mask_zaragoza_four_to_sinker = (
    (team_pitchers['Pitcher'] == 'Zaragoza, Ernesto') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Auto‐retagging {mask_zaragoza_four_to_sinker.sum()} Zaragoza four‐seams (HorzBreak > 10) → Sinker")
team_pitchers.loc[mask_zaragoza_four_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Ernesto Zaragoza: force‐tag Sliders with IVB > 0 → Cutter ——
mask_zaragoza_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Zaragoza, Ernesto') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto‐retagging {mask_zaragoza_slider_to_cutter.sum()} Zaragoza sliders (IVB > 0) → Cutter")
team_pitchers.loc[mask_zaragoza_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Ernesto Zaragoza: force‐tag Sliders with IVB < 0 → Curveball ——
mask_zaragoza_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Zaragoza, Ernesto') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto‐retagging {mask_zaragoza_slider_to_curve.sum()} Zaragoza sliders (IVB < 0) → Curveball")
team_pitchers.loc[mask_zaragoza_slider_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— Ernesto Zaragoza: force‐tag Changeups with RelSpeed > 87.5 → Sinker ——
mask_zaragoza_change_to_sinker = (
    (team_pitchers['Pitcher'] == 'Zaragoza, Ernesto') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 87.5)
)
print(f"Auto‐retagging {mask_zaragoza_change_to_sinker.sum()} Zaragoza changeups (RelSpeed > 87.5) → Sinker")
team_pitchers.loc[mask_zaragoza_change_to_sinker, 'TaggedPitchType'] = 'Sinker'

mask_brown_four = (
    (team_pitchers['Pitcher'] == 'Brown, Tanner') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (
        (team_pitchers['InducedVertBreak'] > 11) |
        (team_pitchers['HorzBreak'] > -9)
    )
)
print(f"Auto‑retagging {mask_brown_four.sum()} Brown sinkers (IVB > 11 or HorzBreak > -9) → Four-Seam")
team_pitchers.loc[mask_brown_four, 'TaggedPitchType'] = 'Four-Seam'

mask_dalen_cutter = (
    (team_pitchers['Pitcher'] == 'Dalen, Kody') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 2.5)
)
print(f"Auto‐retagging {mask_dalen_cutter.sum()} Dalen sliders (IVB > 2.5) → Cutter")
team_pitchers.loc[mask_dalen_cutter, 'TaggedPitchType'] = 'Cutter'

mask_dorminy_change_to_four = (
    (team_pitchers['Pitcher'] == 'Dorminy, Thomas') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 84.5)
)
print(f"Auto‐retagging {mask_dorminy_change_to_four.sum()} Dorminy changeups (RelSpeed > 84.5) → Four-Seam")
team_pitchers.loc[mask_dorminy_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Thomas Dorminy: force‐tag Curveballs with IVB > -11 or HorzBreak > 20 → Slider ——
mask_dorminy_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Dorminy, Thomas') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (
        (team_pitchers['InducedVertBreak'] > -11) |
        (team_pitchers['HorzBreak'] > 20)
    )
)
print(f"Auto‐retagging {mask_dorminy_curve_to_slider.sum()} Dorminy curveballs (IVB > -11 or HorzBreak > 20) → Slider")
team_pitchers.loc[mask_dorminy_curve_to_slider, 'TaggedPitchType'] = 'Slider'

mask_knoll_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Knoll, Brendan') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto‐retagging {mask_knoll_slider_to_curve.sum()} Knoll sliders (IVB < 0) → Curveball")
team_pitchers.loc[mask_knoll_slider_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— Brendan Knoll: force‐tag Sliders, Cutters & Sinkers with IVB > 0 → Four‐Seam ——
mask_knoll_types_to_four = (
    (team_pitchers['Pitcher'] == 'Knoll, Brendan') &
    (team_pitchers['TaggedPitchType'].isin(['Slider','Cutter','Sinker'])) &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto‐retagging {mask_knoll_types_to_four.sum()} Knoll pitches (Slider/Cutter/Sinker & IVB > 0) → Four‐Seam")
team_pitchers.loc[mask_knoll_types_to_four, 'TaggedPitchType'] = 'Four-Seam'

mask_lalonde_splitter = (
    (team_pitchers['Pitcher'] == 'LaLonde, Cole') &
    (team_pitchers['HorzBreak'] > 1) &
    (team_pitchers['InducedVertBreak'] < 10)
)
print(f"Auto‑retagging {mask_lalonde_splitter.sum()} LaLonde pitches (HorzBreak > 1 & IVB < 10) → Splitter")
team_pitchers.loc[mask_lalonde_splitter, 'TaggedPitchType'] = 'Splitter'

mask_levine_four_to_sinker = (
    (team_pitchers['Pitcher'] == 'Levine, Will') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] > 8.5)
)
print(f"Auto‐retagging {mask_levine_four_to_sinker.sum()} Levine four‑seams (IVB > 8.5) → Sinker")
team_pitchers.loc[mask_levine_four_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Will Levine: force‐tag Changeups with RelSpeed > 86 → Sinker ——
mask_levine_change_to_sinker = (
    (team_pitchers['Pitcher'] == 'Levine, Will') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 86)
)
print(f"Auto‐retagging {mask_levine_change_to_sinker.sum()} Levine changeups (RelSpeed > 86) → Sinker")
team_pitchers.loc[mask_levine_change_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Will Levine: force‐tag Sliders with HorzBreak > -1.5 & IVB > 3 → Cutter ——
mask_levine_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Levine, Will') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > -1.5) &
    (team_pitchers['InducedVertBreak'] > 3)
)
print(f"Auto‐retagging {mask_levine_slider_to_cutter.sum()} Levine sliders (HorzBreak > -1.5 & IVB > 3) → Cutter")
team_pitchers.loc[mask_levine_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Will Levine: force‐tag Four‐Seams with IVB < 10 & HorzBreak < 5 → Cutter ——
mask_levine_four_to_cutter = (
    (team_pitchers['Pitcher'] == 'Levine, Will') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['InducedVertBreak'] < 10) &
    (team_pitchers['HorzBreak'] < 5)
)
print(f"Auto‐retagging {mask_levine_four_to_cutter.sum()} Levine four‑seams (IVB < 10 & HorzBreak < 5) → Cutter")
team_pitchers.loc[mask_levine_four_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Will Levine: force‐tag Sliders & Curveballs with HorzBreak < 0 → Sweeper ——
mask_levine_sweeper = (
    (team_pitchers['Pitcher'] == 'Levine, Will') &
    (team_pitchers['TaggedPitchType'].isin(['Slider','Curveball'])) &
    (team_pitchers['HorzBreak'] < 0)
)
print(f"Auto‐retagging {mask_levine_sweeper.sum()} Levine sliders/curveballs (HorzBreak < 0) → Sweeper")
team_pitchers.loc[mask_levine_sweeper, 'TaggedPitchType'] = 'Sweeper'

mask_miller_change_to_sinker = (
    (team_pitchers['Pitcher'] == 'Miller, Seth') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto‐retagging {mask_miller_change_to_sinker.sum()} Miller changeups (RelSpeed > 85) → Sinker")
team_pitchers.loc[mask_miller_change_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Seth Miller: force‐tag Sliders with HorzBreak > -1.5 → Cutter ——
mask_miller_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Miller, Seth') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > -1.5)
)
print(f"Auto‐retagging {mask_miller_slider_to_cutter.sum()} Miller sliders (HorzBreak > -1.5) → Cutter")
team_pitchers.loc[mask_miller_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Seth Miller: force‐tag Sliders & Curveballs with HorzBreak < -1.5 → Sweeper ——
mask_miller_sweeper = (
    (team_pitchers['Pitcher'] == 'Miller, Seth') &
    (team_pitchers['TaggedPitchType'].isin(['Slider','Curveball'])) &
    (team_pitchers['HorzBreak'] < -1.5)
)
print(f"Auto‐retagging {mask_miller_sweeper.sum()} Miller sliders & curveballs (HorzBreak < -1.5) → Sweeper")
team_pitchers.loc[mask_miller_sweeper, 'TaggedPitchType'] = 'Sweeper'

mask_richardson_change_to_four = (
    (team_pitchers['Pitcher'] == 'Richardson, Ryan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 80)
)
print(f"Auto‑retagging {mask_richardson_change_to_four.sum()} Richardson changeups (RelSpeed > 80) → Four-Seam")
team_pitchers.loc[mask_richardson_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

mask_torgerson_change_to_four = (
    (team_pitchers['Pitcher'] == 'Torgerson, Cade') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 83)
)
print(f"Auto‑retagging {mask_torgerson_change_to_four.sum()} Torgerson changeups (RelSpeed > 83) → Four-Seam")
team_pitchers.loc[mask_torgerson_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

mask_washington_four_to_sinker = (
    (team_pitchers['Pitcher'] == 'Washington, Mark') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Auto‑retagging {mask_washington_four_to_sinker.sum()} Washington four‑seams (HorzBreak > 10) → Sinker")
team_pitchers.loc[mask_washington_four_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Mark Washington: force‐tag Curveballs with IVB > -7 → Slider ——
mask_washington_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Washington, Mark') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > -7)
)
print(f"Auto‑retagging {mask_washington_curve_to_slider.sum()} Washington curveballs (IVB > -7) → Slider")
team_pitchers.loc[mask_washington_curve_to_slider, 'TaggedPitchType'] = 'Slider'

mask_chalmers_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Chalmers, Dakota') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < -7)
)
print(f"Auto‑retagging {mask_chalmers_slider_to_curve.sum()} Chalmers sliders (IVB < -7) → Curveball")
team_pitchers.loc[mask_chalmers_slider_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— Dakota Chalmers: force‐tag Curveballs with IVB > -7 → Slider ——
mask_chalmers_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Chalmers, Dakota') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > -7)
)
print(f"Auto‑retagging {mask_chalmers_curve_to_slider.sum()} Chalmers curveballs (IVB > -7) → Slider")
team_pitchers.loc[mask_chalmers_curve_to_slider, 'TaggedPitchType'] = 'Slider'

mask_ligtenberg_change_to_four = (
    (team_pitchers['Pitcher'] == 'Ligtenberg, Reese') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto-retagging {mask_ligtenberg_change_to_four.sum()} Ligtenberg changeups (RelSpeed > 85) → Four-Seam")
team_pitchers.loc[mask_ligtenberg_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

mask_ligtenberg_slider = (
    (team_pitchers['Pitcher'] == 'Ligtenberg, Reese') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (
        (team_pitchers['HorzBreak'] < 0) |
        (team_pitchers['InducedVertBreak'] < 10)
    )
)
print(f"Auto-retagging {mask_ligtenberg_slider.sum()} Ligtenberg changeups (HorzBreak < 0 or IVB < 10) → Slider")
team_pitchers.loc[mask_ligtenberg_slider, 'TaggedPitchType'] = 'Slider'

mask_cavaco_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Cavaco, Keoni') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 80.5)
)
print(f"Auto‐retagging {mask_cavaco_slider_to_cutter.sum()} Cavaco sliders (RelSpeed > 82.5) → Cutter")
team_pitchers.loc[mask_cavaco_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Keoni Cavaco: force‐tag Curveballs with RelSpeed > 74.5 → Slider ——
mask_cavaco_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Cavaco, Keoni') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['RelSpeed'] > 73.0)
)
print(f"Auto‐retagging {mask_cavaco_curve_to_slider.sum()} Cavaco curveballs (RelSpeed > 74.5) → Slider")
team_pitchers.loc[mask_cavaco_curve_to_slider, 'TaggedPitchType'] = 'Slider'

mask_ranaudo_four_to_change = (
    (team_pitchers['Pitcher'] == 'Ranaudo, Anthony') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['RelSpeed'] < 88)
)
print(f"Auto-retagging {mask_ranaudo_four_to_change.sum()} Ranaudo four-seams (RelSpeed < 88) → Changeup")
team_pitchers.loc[mask_ranaudo_four_to_change, 'TaggedPitchType'] = 'Changeup'

# —— Anthony Ranaudo: force‐tag Sliders with IVB > 0 → Cutter ——
mask_ranaudo_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Ranaudo, Anthony') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto-retagging {mask_ranaudo_slider_to_cutter.sum()} Ranaudo sliders (IVB > 0) → Cutter")
team_pitchers.loc[mask_ranaudo_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Anthony Ranaudo: force‐tag Sliders with IVB < 0 → Curveball ——
mask_ranaudo_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Ranaudo, Anthony') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < 0)
)
print(f"Auto-retagging {mask_ranaudo_slider_to_curve.sum()} Ranaudo sliders (IVB < 0) → Curveball")
team_pitchers.loc[mask_ranaudo_slider_to_curve, 'TaggedPitchType'] = 'Curveball'

mask_vallone_four = (
    (team_pitchers['Pitcher'] == 'Vallone, Michael') &
    (team_pitchers['InducedVertBreak'] > 9.5)
)
print(f"Auto‐retagging {mask_vallone_four.sum()} Vallone pitches (IVB > 9.5) → Four‐Seam")
team_pitchers.loc[mask_vallone_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Michael Vallone: force‐tag any pitch with HorzBreak > 0 → Sweeper ——
mask_vallone_sweeper = (
    (team_pitchers['Pitcher'] == 'Vallone, Michael') &
    (team_pitchers['HorzBreak'] > 0)
)
print(f"Auto‐retagging {mask_vallone_sweeper.sum()} Vallone pitches (HorzBreak > 0) → Sweeper")
team_pitchers.loc[mask_vallone_sweeper, 'TaggedPitchType'] = 'Sweeper'

mask_marshall_four = (
    (team_pitchers['Pitcher'] == 'Marshall, Dwayne') &
    (team_pitchers['RelSpeed'] > 86.5)
)
print(f"Auto-retagging {mask_marshall_four.sum()} Marshall pitches (RelSpeed > 86.5) → Four-Seam")
team_pitchers.loc[mask_marshall_four, 'TaggedPitchType'] = 'Four-Seam'

mask_busse_four = (
    (team_pitchers['Pitcher'] == 'Busse, Terry') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['HorzBreak'] < 15.5)
)
print(f"Auto-retagging {mask_busse_four.sum()} Busse pitches (Sinker, HB < 15.5) → Four-Seam")
team_pitchers.loc[mask_busse_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Terry Busse: sliders with RelSpeed > 85 → Cutter ——
mask_busse_cutter = (
    (team_pitchers['Pitcher'] == 'Busse, Terry') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto-retagging {mask_busse_cutter.sum()} Busse pitches (Slider, RelSpeed > 85) → Cutter")
team_pitchers.loc[mask_busse_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Juan Diaz: force-tag Sinkers with IVB > 11 → Four-Seam ——
mask_diaz_sinker_to_four = (
    (team_pitchers['Pitcher'] == 'Diaz, Juan') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 11)
)
print(f"Auto-retagging {mask_diaz_sinker_to_four.sum()} Diaz sinkers (IVB > 11) → Four-Seam")
team_pitchers.loc[mask_diaz_sinker_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Juan Diaz: force-tag Changeups with RelSpeed > 85 → Sinker ——
mask_diaz_change_to_sinker = (
    (team_pitchers['Pitcher'] == 'Diaz, Juan') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto-retagging {mask_diaz_change_to_sinker.sum()} Diaz changeups (RelSpeed > 85) → Sinker")
team_pitchers.loc[mask_diaz_change_to_sinker, 'TaggedPitchType'] = 'Sinker'

# —— Juan Diaz: force-tag Curveballs with IVB > 0 → Slider ——
mask_diaz_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Diaz, Juan') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto-retagging {mask_diaz_curve_to_slider.sum()} Diaz curveballs (IVB > 0) → Slider")
team_pitchers.loc[mask_diaz_curve_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Juan Diaz: force-tag Sliders with IVB < -5 → Curveball ——
mask_diaz_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Diaz, Juan') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < -5)
)
print(f"Auto-retagging {mask_diaz_slider_to_curve.sum()} Diaz sliders (IVB < -5) → Curveball")
team_pitchers.loc[mask_diaz_slider_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— Juan Diaz: force-tag Cutters with RelSpeed < 85 → Slider ——
mask_diaz_cutter_to_slider = (
    (team_pitchers['Pitcher'] == 'Diaz, Juan') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['RelSpeed'] < 85)
)
print(f"Auto-retagging {mask_diaz_cutter_to_slider.sum()} Diaz cutters (RelSpeed < 85) → Slider")
team_pitchers.loc[mask_diaz_cutter_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Matt Givin: force-tag Four-Seams with HB < 10 → Cutter ——
mask_givin_four_to_cutter = (
    (team_pitchers['Pitcher'] == 'Givin, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] < 10)
)
print(f"Auto-retagging {mask_givin_four_to_cutter.sum()} Givin four-seams (HB < 10) → Cutter")
team_pitchers.loc[mask_givin_four_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Matt Givin: force-tag Four-Seams with HB > 10 → Changeup ——
mask_givin_four_to_change = (
    (team_pitchers['Pitcher'] == 'Givin, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Four-Seam') &
    (team_pitchers['HorzBreak'] > 10)
)
print(f"Auto-retagging {mask_givin_four_to_change.sum()} Givin four-seams (HB > 10) → Changeup")
team_pitchers.loc[mask_givin_four_to_change, 'TaggedPitchType'] = 'Changeup'

# —— Matt Givin: force-tag Sliders with HB > -5.5 & IVB > 10 → Cutter ——
mask_givin_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Givin, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['HorzBreak'] > -5.5) &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto-retagging {mask_givin_slider_to_cutter.sum()} Givin sliders (HB > -5.5 & IVB > 0) → Cutter")
team_pitchers.loc[mask_givin_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Matt Givin: force-tag Curveballs with IVB > -3 → Slider ——
mask_givin_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Givin, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['InducedVertBreak'] > -3)
)
print(f"Auto-retagging {mask_givin_curve_to_slider.sum()} Givin curveballs (IVB > -3) → Slider")
team_pitchers.loc[mask_givin_curve_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Matt Givin: force-tag Sliders with IVB < -5 → Curveball ——
mask_givin_slider_to_curve = (
    (team_pitchers['Pitcher'] == 'Givin, Matt') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] < -5)
)
print(f"Auto-retagging {mask_givin_slider_to_curve.sum()} Givin sliders (IVB < -5) → Curveball")
team_pitchers.loc[mask_givin_slider_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— Shawn Mannering: Changeups (RelSpeed > 86) → Four-Seam ——
mask_mannering_ch_to_four = (
    (team_pitchers['Pitcher'] == 'Mannering, Shawn') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 86)
)
print(f"Auto-retagging {mask_mannering_ch_to_four.sum()} Mannering changeups (RelSpeed > 86) → Four-Seam")
team_pitchers.loc[mask_mannering_ch_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Shawn Mannering: Sinkers (IVB > 7) → Four-Seam ——
mask_mannering_sinker_to_four = (
    (team_pitchers['Pitcher'] == 'Mannering, Shawn') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 0)
)
print(f"Auto-retagging {mask_mannering_sinker_to_four.sum()} Mannering sinkers (IVB > 7) → Four-Seam")
team_pitchers.loc[mask_mannering_sinker_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Brady Miller: Sinkers with RelSpeed < 87 → Changeup ——
mask_miller_sinker_to_change = (
    (team_pitchers['Pitcher'] == 'Miller, Brady') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['RelSpeed'] < 87)
)
print(f"Auto-retagging {mask_miller_sinker_to_change.sum()} Miller sinkers (RelSpeed < 87) → Changeup")
team_pitchers.loc[mask_miller_sinker_to_change, 'TaggedPitchType'] = 'Changeup'

# —— Brady Miller: Sinkers with IVB > 13.5 → Four-Seam ——
mask_miller_sinker_to_four = (
    (team_pitchers['Pitcher'] == 'Miller, Brady') &
    (team_pitchers['TaggedPitchType'] == 'Sinker') &
    (team_pitchers['InducedVertBreak'] > 13.5)
)
print(f"Auto-retagging {mask_miller_sinker_to_four.sum()} Miller sinkers (IVB > 13.5) → Four-Seam")
team_pitchers.loc[mask_miller_sinker_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Brady Miller: Sliders with IVB > -0.5 & HB > -6.5 → Cutter ——
mask_miller_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Miller, Brady') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['InducedVertBreak'] > -0.5) &
    (team_pitchers['HorzBreak'] > -6.5)
)
print(f"Auto-retagging {mask_miller_slider_to_cutter.sum()} Miller sliders (IVB > -0.5 & HB > -6.5) → Cutter")
team_pitchers.loc[mask_miller_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Patrick Pridgen: Sliders with RelSpeed > 85 → Cutter ——
mask_pridgen_slider_to_cutter = (
    (team_pitchers['Pitcher'] == 'Pridgen, Patrick') &
    (team_pitchers['TaggedPitchType'] == 'Slider') &
    (team_pitchers['RelSpeed'] > 85)
)
print(f"Auto-retagging {mask_pridgen_slider_to_cutter.sum()} Pridgen sliders (RelSpeed > 85) → Cutter")
team_pitchers.loc[mask_pridgen_slider_to_cutter, 'TaggedPitchType'] = 'Cutter'

# —— Patrick Pridgen: Curveballs with RelSpeed > 78 → Slider ——
mask_pridgen_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Pridgen, Patrick') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['RelSpeed'] > 78)
)
print(f"Auto-retagging {mask_pridgen_curve_to_slider.sum()} Pridgen curveballs (RelSpeed > 78) → Slider")
team_pitchers.loc[mask_pridgen_curve_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Eric Chalus Jr.: Changeups with RelSpeed > 85.5 → Four-Seam ——
mask_chalus_change_to_four = (
    (team_pitchers['Pitcher'] == 'Chalus Jr., Eric') &
    (team_pitchers['TaggedPitchType'] == 'Changeup') &
    (team_pitchers['RelSpeed'] > 85.5)
)
print(f"Auto-retagging {mask_chalus_change_to_four.sum()} Chalus changeups (RelSpeed > 85.5) → Four-Seam")
team_pitchers.loc[mask_chalus_change_to_four, 'TaggedPitchType'] = 'Four-Seam'

# —— Eric Chalus Jr.: Curveballs with RelSpeed > 77.25 → Slider ——
mask_chalus_curve_to_slider = (
    (team_pitchers['Pitcher'] == 'Chalus Jr., Eric') &
    (team_pitchers['TaggedPitchType'] == 'Curveball') &
    (team_pitchers['RelSpeed'] > 77.25)
)
print(f"Auto-retagging {mask_chalus_curve_to_slider.sum()} Chalus curveballs (RelSpeed > 77.25) → Slider")
team_pitchers.loc[mask_chalus_curve_to_slider, 'TaggedPitchType'] = 'Slider'

# —— Eric Chalus Jr.: Sweepers → Curveball ——
mask_chalus_sweeper_to_curve = (
    (team_pitchers['Pitcher'] == 'Chalus Jr., Eric') &
    (team_pitchers['TaggedPitchType'] == 'Sweeper')
)
print(f"Auto-retagging {mask_chalus_sweeper_to_curve.sum()} Chalus sweepers → Curveball")
team_pitchers.loc[mask_chalus_sweeper_to_curve, 'TaggedPitchType'] = 'Curveball'

# —— John Baker: Cutters with RelSpeed > 84.5 → Four-Seam ——
mask_baker_cutter_to_four = (
    (team_pitchers['Pitcher'] == 'Baker, John') &
    (team_pitchers['TaggedPitchType'] == 'Cutter') &
    (team_pitchers['RelSpeed'] > 84.5)
)
print(f"Auto-retagging {mask_baker_cutter_to_four.sum()} Baker cutters (RelSpeed > 84.5) → Four-Seam")
team_pitchers.loc[mask_baker_cutter_to_four, 'TaggedPitchType'] = 'Four-Seam'

unique_pitchers = team_pitchers['Pitcher'].unique()




pitch_colors = {
    'Four-Seam': 'red',
    'Sinker': 'orange',
    'Slider': 'blue',
    'Curveball': 'purple',
    'Changeup': 'green',
    'Cutter': 'cyan',
    'Splitter': 'pink',
    'Knuckleball': 'yellow',
    'Sweeper': 'teal',
}

# === COUNT‐STATE USAGE CONFIG ===
# tuples of (balls, strikes)
ahead_counts  = [(0,1), (1,2), (0,2)]                  # pitcher ahead in count
behind_counts = [(1,0), (2,0), (3,0), (2,1), (3,1)]    # pitcher behind in count

# === PLOT FUNCTIONS ===
def plot_pitch_locations_by_handedness(pitcher_data, pitcher_name, output_dir):
    zone = {'top': 3.673333, 'bottom': 1.524166667, 'left': -0.83083333, 'right': 0.83083333}

    def make_plot(df, handedness, filename):
        import matplotlib.patches as patches
        plt.figure(figsize=(6, 6))
        plt.plot([zone['left'], zone['right']], [zone['top'], zone['top']], 'k-')
        plt.plot([zone['left'], zone['right']], [zone['bottom'], zone['bottom']], 'k-')
        plt.plot([zone['left'], zone['left']], [zone['bottom'], zone['top']], 'k-')
        plt.plot([zone['right'], zone['right']], [zone['bottom'], zone['top']], 'k-')
        for pt in df['TaggedPitchType'].dropna().unique():
            sub = df[df['TaggedPitchType'] == pt]
            c = pitch_colors.get(pt, 'gray')
            x = sub['PlateLocSide'].values
            y = sub['PlateLocHeight'].values
            mx, my = x.mean(), y.mean()
            cov = np.cov(x, y)
            vals, vecs = np.linalg.eigh(cov)
            order = vals.argsort()[::-1]
            vals, vecs = vals[order], vecs[:, order]
            width, height = 2 * np.sqrt(vals[0]), 2 * np.sqrt(vals[1])
            angle = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))
            ellipse = patches.Ellipse((mx, my), width, height, angle=angle,
                                      edgecolor=c, facecolor=c, alpha=0.2)
            plt.gca().add_patch(ellipse)
            count = len(sub); total = len(df)
            size = 300 * (count / total)
            plt.scatter(mx, my, color=c, edgecolors='black', s=size, label=f"{pt} ({count})")
        plt.title(f'{pitcher_name} - Pitch Locations vs {handedness}')
        plt.xlim(-2.5, 2.5); plt.ylim(0, 5)
        plt.xlabel('Horizontal Location (ft)'); plt.ylabel('Vertical Location (ft)')
        plt.legend(fontsize=8, loc='lower left'); plt.grid(True, linestyle='--', alpha=0.4)
        plt.tight_layout(); plt.savefig(filename); plt.close()

    os.makedirs(output_dir, exist_ok=True)
    make_plot(pitcher_data[pitcher_data['BatterSide']=='Left'], 'LHB',
              os.path.join(output_dir, f"{pitcher_name.replace(',','')}_zone_lhb.png"))
    make_plot(pitcher_data[pitcher_data['BatterSide']=='Right'], 'RHB',
              os.path.join(output_dir, f"{pitcher_name.replace(',','')}_zone_rhb.png"))


def plot_pitch_movement(pitcher_data, pitcher_name, output_dir):
    plt.figure(figsize=(6, 6))
    L = 35
    for pt in pitcher_data['TaggedPitchType'].dropna().unique():
        sub = pitcher_data[pitcher_data['TaggedPitchType']==pt]
        plt.scatter(sub['HorzBreak'], sub['InducedVertBreak'], label=pt,
                    color=pitch_colors.get(pt, 'gray'), alpha=0.6, s=80, edgecolors='black')
        if False and 'ThrowPositionX' in sub.columns and 'ThrowPositionZ' in sub.columns and not sub.empty:
            ax = sub['ThrowPositionX'].median(); az = sub['ThrowPositionZ'].median()
            if az>0 and abs(ax)>=1e-3:
                ang = np.arctan2(ax, az)
                ex, ey = L*np.sin(ang), L*np.cos(ang)
                ex = abs(ex) if pitcher_data['PitcherThrows'].iloc[0].upper().startswith('R') else -abs(ex)
                plt.plot([0,ex],[0,ey],'--',linewidth=2,color=pitch_colors.get(pt,'gray'))
    plt.axhline(0, color='gray', linestyle='--'); plt.axvline(0, color='gray', linestyle='--')
    plt.title(f'{pitcher_data["Pitcher"].iloc[0]} - Pitch Movement')
    plt.xlabel('Horizontal Break (in)'); plt.ylabel('Induced Vertical Break (in)')
    plt.xlim(-30,30); plt.ylim(-30,30); plt.grid(True, linestyle='--', alpha=0.4)
    plt.legend(fontsize=8, loc='lower right'); plt.tight_layout()
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, f"{pitcher_name.replace(',','')}_pitch_movement.png")); plt.close()


def plot_pitch_outcomes_table(pitcher_data, pitcher_name, output_dir):
    plt.rcParams['font.family'] = 'Helvetica'
    summary = pitcher_data.groupby('TaggedPitchType').agg(
        Count=('PitchNo','count'),
        UsagePercent=('PitchNo', lambda x: len(x)/len(pitcher_data)*100),
        Velocity=('RelSpeed','mean'),
        HB=('HorzBreak','mean'),
        IVB=('InducedVertBreak','mean'),
        SpinRate=('SpinRate','mean'),
        ZonePercent=('PlateLocHeight', lambda x: ((pitcher_data.loc[x.index,'PlateLocHeight'].between(1.524,3.673)) & (pitcher_data.loc[x.index,'PlateLocSide'].abs()<=0.831)).mean()*100),
        WhiffPercent=('PitchCall', lambda x: ((x=='StrikeSwinging').sum()/x.isin(['StrikeSwinging','FoulBallFieldable','FoulBallNotFieldable','InPlay']).sum()*100) if x.isin(['StrikeSwinging','FoulBallFieldable','FoulBallNotFieldable','InPlay']).sum()>0 else 0),
        ChasePercent=('PitchCall', lambda x: (((~((pitcher_data.loc[x.index,'PlateLocHeight'].between(1.524,3.673))&(pitcher_data.loc[x.index,'PlateLocSide'].abs()<=0.831)))&x.isin(['StrikeSwinging','FoulBallFieldable','FoulBallNotFieldable','InPlay'])).sum()/((~((pitcher_data.loc[x.index,'PlateLocHeight'].between(1.524,3.673))&(pitcher_data.loc[x.index,'PlateLocSide'].abs()<=0.831)))).sum()*100) if ((~((pitcher_data.loc[x.index,'PlateLocHeight'].between(1.524,3.673))&(pitcher_data.loc[x.index,'PlateLocSide'].abs()<=0.831)))).sum()>0 else 0),
        VAA=('VertApprAngle','mean'),
        HAA=('HorzApprAngle','mean'),
    ).round(1).reset_index()
    summary = summary.rename(columns={'TaggedPitchType':'Pitch Type','UsagePercent':'Usage%','Velocity':'Velocity','ZonePercent':'Zone%','WhiffPercent':'Whiff%','ChasePercent':'Chase%'})
    summary = summary.sort_values('Usage%',ascending=False).reset_index(drop=True)
    plt.figure(figsize=(10,len(summary)*0.6+1)); plt.axis('off')
    tbl=plt.table(cellText=summary.values,colLabels=summary.columns,loc='center',cellLoc='center')
    tbl.auto_set_font_size(False); tbl.set_fontsize(9); tbl.scale(1.2,1.5)
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, f"{pitcher_name.replace(',','')}_pitch_outcomes.png"),bbox_inches='tight'); plt.close()


def plot_velocity_distribution(pitcher_data, pitcher_name, output_dir):
    sns.set_style('whitegrid')
    pitch_types=sorted(pitcher_data['TaggedPitchType'].unique())
    n=len(pitch_types)
    fig_height=max(2,n*0.6)
    fig,axes=plt.subplots(n,1,sharex=True,figsize=(5,fig_height),dpi=100)
    if n==1: axes=[axes]
    for ax,pt in zip(axes,pitch_types):
        data=pitcher_data.loc[pitcher_data['TaggedPitchType']==pt,'RelSpeed']
        if data.empty: continue
        sns.kdeplot(data,fill=True,ax=ax,alpha=0.6,linewidth=1.2)
        ax.axvline(data.mean(),color='k',linestyle='--',linewidth=0.8)
        ax.set_ylabel(pt,rotation=0,labelpad=25,va='center',fontsize=8)
        ax.set_yticks([]); ax.tick_params(axis='x',labelsize=7)
    axes[-1].set_xlabel('Velocity (mph)',fontsize=8)
    fig.suptitle('Pitch Velocity Distribution',y=1.01,fontsize=10)
    plt.tight_layout(rect=[0,0,1,0.95],h_pad=0.4)
    os.makedirs(output_dir, exist_ok=True)
    out_path=os.path.join(output_dir, f"{pitcher_name.replace(',','')}_velocity_dist.png")
    fig.savefig(out_path,bbox_inches='tight',dpi=100); plt.close(fig)
    return out_path

def plot_usage_by_count(pitcher_data, pitcher_name, output_dir):
    """
    Builds a mirrored bar chart of pitch‐type usage:
      • Left = usage when ahead in count
      • Right = usage when behind in count
    """

    # build a Series of tuples for each pitch
    # adjust 'Balls'/'Strikes' to match your actual column names
    counts = list(zip(pitcher_data['Balls'], pitcher_data['Strikes']))

    # mask for ahead vs behind
    mask_ahead  = [c in ahead_counts  for c in counts]
    mask_behind = [c in behind_counts for c in counts]

    # compute usage%
    use_a = (pitcher_data.loc[mask_ahead,  'TaggedPitchType']
             .value_counts(normalize=True) * 100)
    use_b = (pitcher_data.loc[mask_behind, 'TaggedPitchType']
             .value_counts(normalize=True) * 100)

    # ensure all pitch types are present
    all_types = sorted(set(use_a.index) | set(use_b.index))
    use_a = use_a.reindex(all_types, fill_value=0)
    use_b = use_b.reindex(all_types, fill_value=0)

    # prepare DataFrame for plotting
    df = pd.DataFrame({
        'Ahead (%)':  -use_a,   # negative goes left
        'Behind (%)': use_b
    }, index=all_types)

    # plot
    fig, ax = plt.subplots(figsize=(6, max(2, len(all_types)*0.5)))
    y_pos = np.arange(len(all_types))

    for idx, pt in enumerate(all_types):
        val_a = use_a[pt]
        val_b = use_b[pt]
        c     = pitch_colors.get(pt, 'gray')

        # ahead (left, negative)
        ax.barh(y_pos[idx], -val_a, color=c, edgecolor='black')
        # behind (right, positive)
        ax.barh(y_pos[idx],  val_b, color=c, edgecolor='black')
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda x, pos: f"{abs(x):.0f}")
    )

    ax.set_yticks(y_pos)
    ax.set_yticklabels(all_types)
    ax.axvline(0, color='k', linewidth=1)
    ax.set_xlabel('Usage %')
    ax.set_title(f'{pitcher_name} – Usage by Count State', pad=20)

    ax.text(0.25, 1.02, 'Ahead in Count',
            ha='center', va='bottom',
            transform=ax.transAxes, fontsize=9)
    ax.text(0.75, 1.02, 'Behind in Count',
            ha='center', va='bottom',
            transform=ax.transAxes, fontsize=9)

    plt.tight_layout()
    # ————————————————————————————————————————

    # save as before
    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(
        output_dir,
        f"{pitcher_name.replace(',','')}_usage_count.png"
    )
    fig.savefig(out_path, bbox_inches='tight')
    plt.close(fig)
    return out_path


# === PDF GENERATION ===
def generate_clean_pdf(pitcher_name,pitcher_data,output_dir):
    zone_l=os.path.join(output_dir,f"{pitcher_name.replace(',','')}_zone_lhb.png")
    zone_r=os.path.join(output_dir,f"{pitcher_name.replace(',','')}_zone_rhb.png")
    pmov=os.path.join(output_dir,f"{pitcher_name.replace(',','')}_pitch_movement.png")
    pout=os.path.join(output_dir,f"{pitcher_name.replace(',','')}_pitch_outcomes.png")
    vdist=os.path.join(output_dir,f"{pitcher_name.replace(',','')}_velocity_dist.png")

    class CustomPDF(FPDF):
        def header(self):
            team_code = pitcher_data['PitcherTeam'].iloc[0] if 'PitcherTeam' in pitcher_data.columns else 'default'
            base_path=os.getcwd()
            logo_path=os.path.join(base_path,'assets',f'{team_code}_logo.png')
            if os.path.exists(logo_path): self.image(logo_path,x=10,y=14,w=35)
            self.set_font('Helvetica','',16)
            ph=pitcher_data['PitcherThrows'].iloc[0] if 'PitcherThrows' in pitcher_data.columns else ''
            hl='LHP' if ph.upper().startswith('L') else 'RHP' if ph.upper().startswith('R') else ph
            self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
            self.set_font('Helvetica','',12)
            self.cell(0,8,'2025 American Association Season',ln=True,align='C')
            headshot=os.path.join(base_path,'headshots',f"{pitcher_name.replace(', ','_')}_headshot.png")
            if os.path.exists(headshot): self.image(headshot,x=170,y=8,w=30)
            self.ln(5)

        def add_image_section(self,title,image_path):
            self.set_font('Arial','B',12); self.cell(0,8,title,ln=True)
            if os.path.exists(image_path): self.image(image_path,w=180)
            self.ln(5)

    pdf=CustomPDF(orientation='P',unit='mm',format='A4')
    pdf.set_auto_page_break(False, margin=10)
    pdf.add_page(); pdf.set_y(55)
# === STAT LINE ===
    # Calculate innings pitched
    ip_outs = (pitcher_data['OutsOnPlay'].fillna(0).astype(int) +
               pitcher_data['KorBB'].eq('Strikeout').astype(int)).sum()
    ip_whole = ip_outs // 3
    ip_remainder = ip_outs % 3
    ip = ip_whole + (0.1 if ip_remainder == 1 else 0.2 if ip_remainder == 2 else 0)

    # Counting stats
    k = (pitcher_data['KorBB'] == 'Strikeout').sum()
    bb = (pitcher_data['KorBB'] == 'Walk').sum()
    hbp = (pitcher_data['PitchCall'] == 'HitByPitch').sum()
    hr = (pitcher_data['PlayResult'] == 'HomeRun').sum()
    hits = pitcher_data['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun']).sum()
    er = pitcher_data['RunsScored'].sum() if 'RunsScored' in pitcher_data.columns else 0
    # Plate appearances
    pa = ((pitcher_data['OutsOnPlay'].fillna(0).astype(int) +
           pitcher_data['KorBB'].isin(['Strikeout', 'Walk']).astype(int) +
           (pitcher_data['PitchCall'] == 'HitByPitch').astype(int) +
           pitcher_data['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun']).astype(int)).sum())

    # Rate stats
    whip = (bb + hits) / ip if ip else 0
    era = (er * 9) / ip if ip else 0
    # Properly define K% and BB%
    k_pct = (k / pa * 100) if pa else 0
    bb_pct = (bb / pa * 100) if pa else 0

    # Build table row
    pdf.set_font('Arial', '', 11)
    col_labels = ['IP', 'ERA', 'WHIP', 'K%', 'BB%', 'HBP', 'HR', 'Strike%', 'Whiff%']
    row_data = [[
        f"{ip:.1f}",
        f"{era:.2f}",
        f"{whip:.2f}",
        f"{k_pct:.1f}%",
        f"{bb_pct:.1f}%",
        str(hbp),
        str(hr),
        f"{(pitcher_data['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled', 'FoulBallFieldable', 'FoulBallNotFieldable']).sum() / len(pitcher_data) * 100):.1f}%",
        f"{(pitcher_data['PitchCall'].eq('StrikeSwinging').sum() / pitcher_data['PitchCall'].isin(['StrikeSwinging', 'FoulBallFieldable', 'FoulBallNotFieldable', 'InPlay']).sum() * 100 if pitcher_data['PitchCall'].isin(['StrikeSwinging', 'FoulBallFieldable', 'FoulBallNotFieldable', 'InPlay']).sum() > 0 else 0):.1f}%"
    ]]


    col_widths = [18, 22, 25, 15, 15, 18, 15, 20, 18]
    table_y = pdf.get_y()
    table_width = sum(col_widths)
    page_width = pdf.w - 2 * pdf.l_margin
    table_x = (page_width - table_width) / 2 + pdf.l_margin
    table_width = sum(col_widths)
    page_width = pdf.w - 2 * pdf.l_margin
    table_x = (page_width - table_width) / 2 + pdf.l_margin
    

    pdf.set_font('Arial', 'B', 11)
    for i, label in enumerate(col_labels):
        pdf.set_xy(table_x + sum(col_widths[:i]), table_y)
        pdf.cell(col_widths[i], 8, label, border=1, align='C')

    pdf.set_font('Arial', '', 11)
    for j, item in enumerate(row_data[0]):
        pdf.set_xy(table_x + sum(col_widths[:j]), table_y + 8)
        pdf.cell(col_widths[j], 8, item, border=1, align='C')

    pdf.set_y(table_y + 18)
    pdf.cell(0, 8, 'Pitch Location & Movement', ln=True, align='C')
    pdf.ln(2)
    # Plot images
    y0=pdf.get_y()
    for x_pos,img in zip([15,75,135],[zone_l,pmov,zone_r]):
        if os.path.exists(img): pdf.image(img,x=x_pos,y=y0,w=55)
    pdf.ln(60)
    pdf.add_image_section('Pitch Outcomes & Characteristics', pout)
    # ——— side-by-side: Usage by Count (left) & Velocity Dist (right) ———
    usable_width = pdf.w - 2*pdf.l_margin
    gutter = 10                              # space between images
    img_w = (usable_width - gutter) / 2     # each image width

    x_left  = pdf.l_margin
    x_right = pdf.l_margin + img_w + gutter
    y0 = pdf.get_y()

    # 1) Usage‐by‐Count image
    count_img = os.path.join(
        output_dir,
        f"{pitcher_name.replace(',','')}_usage_count.png"
    )
    if os.path.exists(count_img):
        pdf.image(count_img, x=x_left,  y=y0, w=img_w)

    # 2) Velocity‐distribution image
    if os.path.exists(vdist):
        pdf.image(vdist,    x=x_right, y=y0, w=img_w)

    # advance cursor below both images
    pdf.ln( max(img_w*0.6, 60) )
    # ——————————————————————————————————————————————————————————————
    out_folder=os.path.join(output_dir,'season_reports'); os.makedirs(out_folder,exist_ok=True)
    pdf.output(os.path.join(out_folder,f"{pitcher_name.replace(',','')}_season_scout.pdf"))

# === MAIN LOOP ===
for pitcher_name in unique_pitchers:
    pitcher_data=team_pitchers[team_pitchers['Pitcher']==pitcher_name]
    if pitcher_data.empty: continue
    counts=pitcher_data['TaggedPitchType'].value_counts(normalize=True)*100
    valid=counts[counts>=1.5].index.tolist()
    pitcher_data=pitcher_data[pitcher_data['TaggedPitchType'].isin(valid)]
    p_dir=os.path.join(save_folder_path,pitcher_name.replace(',',''))
    os.makedirs(p_dir,exist_ok=True)
    plot_velocity_distribution(pitcher_data,pitcher_name,p_dir)
    plot_pitch_locations_by_handedness(pitcher_data,pitcher_name,p_dir)
    plot_pitch_movement(pitcher_data,pitcher_name,p_dir)
    plot_pitch_outcomes_table(pitcher_data,pitcher_name,p_dir)
    # –– plot count‐state usage and capture file path
    count_img = plot_usage_by_count(pitcher_data, pitcher_name, p_dir)
    generate_clean_pdf(pitcher_name,pitcher_data,p_dir)

    
team_pitchers.to_csv('team_pitchers_tagged.csv', index=False)

Auto-retagging 0 Bourassa curveballs → sliders (IVB > -7.5)
Strobel: retagging 0 sliders → sweepers
Strobel: retagging 0 curveballs → sliders
Auto-retagging 0 Galindo four-seams → sinkers (IVB < 11)
Auto-retagging 0 Bourassa changeups → sinkers (RelSpeed > 85)
Auto-retagging 0 Cherry sinkers → cutters (HorzBreak < 10)
Auto-retagging 0 Echevarria curveballs → sweepers
Overriding 0 rows for Chalus Jr., Eric:sinker
Auto‐retagging 0 Cepeda sinkers (IVB > 10) → Four‐Seam
Dropping 0 Gercken pitches (HorzBreak < -14) from report
Auto‐retagging 0 Gercken four-seams (HorzBreak > 10) → Sinker
Dropping 0 Gercken pitches (IVB < 0 & HorzBreak > 10) from report
Auto-retagging 0 O’Donnell changeups (IVB > 10) → Four-Seam
Auto-retagging 0 O’Donnell changeups (IVB < 10) → Sinker
Auto-retagging 0 O’Donnell four-seams (IVB < 5) → Sinker
Auto‐retagging 0 Scholten pitches (–3 ≤ HorzBreak ≤ 10) → Cutter
Auto‐retagging 0 Drury sliders (–5 < HorzBreak < 7) → Cutter
Auto‐retagging 0 Drury pitches (HorzBreak > 

Auto-retagging 0 Ligtenberg changeups (RelSpeed > 85) → Four-Seam
Auto-retagging 0 Ligtenberg changeups (HorzBreak < 0 or IVB < 10) → Slider
Auto‐retagging 228 Cavaco sliders (RelSpeed > 82.5) → Cutter
Auto‐retagging 87 Cavaco curveballs (RelSpeed > 74.5) → Slider
Auto-retagging 2 Ranaudo four-seams (RelSpeed < 88) → Changeup
Auto-retagging 35 Ranaudo sliders (IVB > 0) → Cutter
Auto-retagging 6 Ranaudo sliders (IVB < 0) → Curveball
Auto‐retagging 206 Vallone pitches (IVB > 9.5) → Four‐Seam
Auto‐retagging 97 Vallone pitches (HorzBreak > 0) → Sweeper
Auto-retagging 494 Marshall pitches (RelSpeed > 86.5) → Four-Seam
Auto-retagging 0 Busse pitches (Sinker, HB < 15.5) → Four-Seam
Auto-retagging 0 Busse pitches (Slider, RelSpeed > 85) → Cutter
Auto-retagging 0 Diaz sinkers (IVB > 11) → Four-Seam
Auto-retagging 0 Diaz changeups (RelSpeed > 85) → Sinker
Auto-retagging 0 Diaz curveballs (IVB > 0) → Slider
Auto-retagging 0 Diaz sliders (IVB < -5) → Curveball
Auto-retagging 0 Diaz cutters (RelSpe

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2131: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,8,'2025 American Association Season',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2173: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 11)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2198: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon 

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2131: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,8,'2025 American Association Season',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2173: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 11)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2198: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon 

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2131: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,8,'2025 American Association Season',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2173: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 11)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2198: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon 

/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:316: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:316: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:1942: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov = np.cov(x, y)
/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/var

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2131: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,8,'2025 American Association Season',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2173: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 11)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2198: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon 

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2131: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,8,'2025 American Association Season',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2173: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 11)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2198: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon 

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:1942: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov = np.cov(x, y)
/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2129: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0,10,f'{pitcher_name} ({hl})',ln=True,align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30396/2974809693.py:2131: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.